In [1]:
from rucio.client.client import Client
import os

import logging
logger = logging.getLogger(__name__)
logging.disable('WARNING')

os.popen("cp /opt/workspace/persistent-storage/proxy .")

os.environ["RUCIO_HOME"] = "/cvmfs/cms.cern.ch/rucio/current/"
os.environ['X509_CERT_DIR'] = "/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
os.environ['X509_USER_PROXY'] = "/opt/workspace/persistent-storage/UL/RDF-VBS_SSWW_lep_tauh/proxy"
RUCIO_SCOPE = "cms"
ACCOUNT = "ttedesch"

SITENAME = "T2_IT_Legnaro"

c = Client(account=ACCOUNT, auth_type="x509_proxy")

In [2]:
#!davix-ls -l davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms/store/user/ttedesch/preselection_UL2017_v24_monitoring_remote_dataNoRunGraphs_v2 -E /opt/workspace/persistent-storage/proxy --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/

In [3]:
%%capture

import ROOT
import os
from dask.distributed import Client
from utils import *
from variablesULbenchmark import *
from samplesUL import *
ROOT.RDF.Experimental.Distributed.open_files_locally = False

variables_to_plot = [#"leadjet_pt", 
                     #"subleadjet_pt",
                     "m_jj",
                     "lepton_pt", "tau_pt", #"MET_pt"
                    ]

redirector = "stormgf2.pi.infn.it"
#results_dir = "postselection_bench_v24_lep_tau_mjj"
#results_dir = "postselection_bench_v24_lep_tau_mjj_bis" OK
#results_dir = "postselection_bench_v24_lep_tau_mjj_data"
#results_dir = "postselection_bench_v24_lep_tau_mjj_data"
#results_dir = "postselection_bench_v24_lep_tau_mjj_O2" OK but network high
#results_dir = "postselection_bench_v24_lep_tau_mjj_O2_v2"
#results_dir = "postselection_bench_v24_lep_tau_mjj_O2_v2_allMC_v1"
#results_dir = "postselection_bench_v24_lep_tau_mjj_O2_v2_allMC_fix_fix_METfix" VALIDATO!!
#results_dir = "postselection_bench_v24_lep_tau_mjj_O2_v2_allMC_fix_fix_METfix_iter1"
#results_dir = "postselection_bench_v24_lep_tau_mjj_O2_v2_allMC_fix_fix_METfix_iter2"
#results_dir = "postselection_bench_v24_lep_tau_mjj_O2_v2_allMC_fix_fix_METfix_iter3"
results_dir = "postselection_bench_v24_lep_tau_mjj_O2_v2_allMC_fix_fix_METfix_iter3_inverted"



#results_dir = "postselection_bench_v24_snap_O2_v2"
#results_dir = "postselection_bench_v24_snap_O2_v3"
#results_dir = "postselection_bench_v24_snap_O2_allMC_v1"
#results_dir = "postselection_bench_v24_snap_O2_allMC_v1_iter3"


#results_dir = "postselection_bench_v24_snap"

remote_folder_name = results_dir

e2e = True
#ERA OK QUELLA SEMPRE USATA folder_e2e = "preselection_UL2017_v24_monitoring_remote"
folder_e2e = "preselection_UL2017_v24_monitoring_remote_v2_O2_iter6"

folder_e2e_data = "preselection_UL2017_v24_monitoring_remote_dataNoRunGraphs_v2"


maxNfilespersample = 99999999

MT = False

do_variations = True
distributed = True
#nmaxpartitions = 92*3
nmaxpartitions = 92*3

sched_port = 26162

outTreeName = "events_nominal"
outFilePath = "./postselectionUL.root"
outFilePath_WZ = "./postselectionUL_WZ.root"
outFilePath_Data = "./postselectionUL_Data.root"

In [4]:
os.popen("davix-mkdir davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms/store/user/ttedesch/{} -E /opt/workspace/persistent-storage/proxy --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name))
def transfer_to_tier(dask_worker):
    import os
    #os.popen("bash transfer_to_tier.sh")
    #return (os.getcwd())
    os.popen('for filename in ./postselectionUL_*.root; do davix-put "$filename" davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms/store/user/ttedesch/{}/"$filename" -E ./proxy --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/; done'.format(remote_folder_name))
    os.popen('for filename in ./*monitor*.csv; do davix-put "$filename" davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms/store/user/ttedesch/{}/"$filename" -E ./proxy --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/; done'.format(remote_folder_name))
    os.popen('for filename in ./*runtime*.csv; do davix-put "$filename" davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms/store/user/ttedesch/{}/"$filename" -E ./proxy --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/; done'.format(remote_folder_name))
    return True

In [5]:
if distributed != True and MT == True:
    ROOT.ROOT.EnableImplicitMT()

# Import utils functions

In [6]:
from distributed.diagnostics.plugin import UploadFile

def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    print(working_dir)
    os.environ['X509_USER_PROXY'] = working_dir + '/proxy'
    os.system("export X509_USER_PROXY={}".format(working_dir + '/proxy'))
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    os.system("export X509_CERT_DIR=/cvmfs/grid.cern.ch/etc/grid-security/certificates/")
    #os.popen('cp {} {}'.format(working_dir + '/proxy', '../../../proxy'))
    shutil.copyfile(working_dir + '/proxy', working_dir + '/../../../proxy')
    
    os.environ['EXTRA_CLING_ARGS'] = "-O2"
    os.system('export EXTRA_CLING_ARGS="-O2"')    
    
    return os.environ.get("X509_USER_PROXY")

In [7]:
text_file = open("postselection_UL.h", "r")
data = text_file.read()
def my_initialization_function():
    
    print(ROOT.gInterpreter.ProcessLine(".O"))
    ROOT.gInterpreter.Declare('{}'.format(data))
    
    from urllib import request
    
    print("creating TMVA object")
    
    modelName = "optimized_model_SM_allBKGs_UL_vUL025_optimization_balancing_redoAN_reduced_simplified"
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/models/{}.hxx".format(modelName), "{}.hxx".format(modelName))
    request.urlretrieve("https://vbs-pg-support.web.cern.ch/models/{}.dat".format(modelName), "{}.dat".format(modelName))
    ROOT.gInterpreter.Declare('#include "' + modelName + '.hxx"')
    ROOT.gInterpreter.Declare('auto sofie_functor = TMVA::Experimental::SofieFunctor<28,TMVA_SOFIE_'+modelName+'::Session>(0);')
    
    print("end of initialization")
    
    
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    client.restart()
    client.register_worker_plugin(UploadFile("./proxy"))
    client.run(set_proxy)
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)

else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

/usr/local/lib/python3.8/dist-packages/distributed/client.py:1128: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| msgpack | 1.0.3  | 1.0.2     | 1.0.3   |
| toolz   | 0.11.2 | 0.11.1    | 0.11.2  |
+---------+--------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


0
creating TMVA object
end of initialization


input_line_52:886:1: warning: control may reach end of non-void function [-Wreturn-type]
}
^


# Define analysis steps

In [8]:
def filter_trigger(df):
    #df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter", "Good vertex selection")
    #df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger") #RR2017
    #df_trigger = df_goodvertex.Filter("HLT_IsoMu24 || HLT_IsoTkMu24 ||  HLT_Ele27_WPTight_Gsf", "Leptonic trigger") ##UL2016*
    df_trigger = df.Filter("HLT_IsoMu27 ||  HLT_Ele35_WPTight_Gsf", "Leptonic trigger") ##UL2017
    #df_trigger = df_goodvertex.Filter("HLT_IsoMu24 ||  HLT_Ele32_WPTight_Gsf", "Leptonic trigger") ##UL2018

    return df_trigger

def select_jets(df):
    df_atleast2Jets = df.Filter("nJet>=2", "At least two jets")
    df_GoodJets = df_atleast2Jets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
    df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
    df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_invmass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, GoodJets_idx)")
    df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets")
    df_jetsDefinitions = df_2VBSjets.Define("leadjet_pt", "GetLeading(Jet_pt, VBSJet_idx)")\
                                    .Define("leadjet_eta", "GetLeading(Jet_eta, VBSJet_idx)")\
                                    .Define("leadjet_phi", "GetLeading(Jet_phi, VBSJet_idx)")\
                                    .Define("leadjet_mass", "GetLeading(Jet_mass, VBSJet_idx)")\
                                    .Define("leadjet_DeepFlv_b", "GetLeading(Jet_btagDeepFlavB, VBSJet_idx)")\
                                    .Define("subleadjet_pt", "GetSubLeading(Jet_pt, VBSJet_idx)")\
                                    .Define("subleadjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)")\
                                    .Define("subleadjet_phi", "GetSubLeading(Jet_phi, VBSJet_idx)")\
                                    .Define("subleadjet_mass", "GetSubLeading(Jet_mass, VBSJet_idx)")\
                                    .Define("subleadjet_DeepFlv_b", "GetSubLeading(Jet_btagDeepFlavB, VBSJet_idx)")\

    return df_jetsDefinitions

def select_lepton(df):
    df_selectElectron = df.Define("Electron_idx", "SelectElectron(Electron_pt, Electron_eta, Electron_phi, Electron_jetRelIso, Electron_mvaFall17V2Iso_WPL, Electron_mvaFall17V2Iso_WP90, Jet_eta, Jet_phi, VBSJet_idx, Year)")
    df_selectMuon = df_selectElectron.Define("Muon_idx", "SelectMuon(Muon_corrected_pt, Muon_eta, Muon_phi, Muon_tightId, Muon_looseId, Muon_pfRelIso04_all, Jet_eta, Jet_phi, VBSJet_idx)")
    df_atLeast1Lepton = df_selectMuon.Filter("Electron_idx[1] != -1 || Muon_idx[1] != -1", "At least 1 at-least-loose lepton")
    df_goodLeptonFamily = df_atLeast1Lepton.Define("GoodLeptonFamily", "DetermineGoodLepton(HLT_IsoMu27, HLT_Mu50, HLT_Ele35_WPTight_Gsf, HLT_Ele32_WPTight_Gsf_L1DoubleEG, HLT_Photon200, HLT_PFHT250, HLT_PFHT350, Electron_idx, Electron_pt, Electron_eta, Electron_mvaFall17V2Iso_WPL, Electron_jetRelIso, Muon_idx, Muon_corrected_pt, Muon_eta, Muon_pfRelIso04_all, Muon_looseId)")
    df_compatibleLeptons = df_goodLeptonFamily.Filter("GoodLeptonFamily != -1 ", "Filter on leptons")
    df_leptonDefinitions = df_compatibleLeptons.Define("lepton_pt", "GetLepton(Electron_pt, Electron_idx, Muon_corrected_pt, Muon_idx, GoodLeptonFamily)")\
                                                .Define("lepton_eta", "GetLepton(Electron_eta, Electron_idx, Muon_eta, Muon_idx, GoodLeptonFamily)")\
                                                .Define("lepton_phi", "GetLepton(Electron_phi, Electron_idx, Muon_phi, Muon_idx, GoodLeptonFamily)")\
                                                .Define("lepton_mass", "GetLepton(Electron_mass, Electron_idx, Muon_mass, Muon_idx, GoodLeptonFamily)")\
                                                .Define("lepton_charge", "GetLepton(Electron_charge, Electron_idx, Muon_charge, Muon_idx, GoodLeptonFamily)")\
                                                .Define("lepton_TightRegion", "GetLeptonTightFlag(Electron_idx, Muon_idx, GoodLeptonFamily)")\
                                                .Define("lepton_LnTRegion", "1 - lepton_TightRegion")\
                                                .Define("lepton_Zeppenfeld", "Zeppenfeld(lepton_eta, leadjet_eta, subleadjet_eta)")\
                                                .Define("lepton_pdgid", "GetLepton(Electron_pdgId, Electron_idx, Muon_pdgId, Muon_idx, GoodLeptonFamily)")\
                                                .Define("lepton_pfRelIso04", "Get_isolation(Electron_jetRelIso, Electron_idx, Muon_pfRelIso04_all, Muon_idx, GoodLeptonFamily)")

    return df_leptonDefinitions

def select_tau(df):
    df_selectTau = df.Define("Tau_idx", "SelectAndVetoTaus(Tau_pt, Tau_eta, Tau_phi, Tau_idDeepTau2017v2p1VSjet, Tau_idDeepTau2017v2p1VSe, Tau_idDeepTau2017v2p1VSmu, GoodLeptonFamily, Electron_idx, Electron_eta, Electron_phi, Muon_idx, Muon_eta, Muon_phi, Jet_eta, Jet_phi, VBSJet_idx, Year)")
    df_1tau = df_selectTau.Filter("Tau_idx[1] != -1", "Exactly 1 at least loose Tau")
    df_tauDefinitions = df_1tau.Define("tau_pt", "GetTau(Tau_pt, Tau_idx)")\
                               .Define("tau_eta", "GetTau(Tau_eta, Tau_idx)")\
                               .Define("tau_phi", "GetTau(Tau_phi, Tau_idx)")\
                               .Define("tau_mass", "GetTau(Tau_mass, Tau_idx)")\
                               .Define("tau_charge", "GetTau(Tau_charge, Tau_idx)")\
                               .Define("tau_DecayMode", "GetTau(Tau_decayMode, Tau_idx)")\
                               .Define("tau_isolation", "GetTau(Tau_neutralIso, Tau_idx)")\
                               .Define("tau_DeepTau_WP", "pow(GetTau(Tau_idDeepTau2017v2p1VSjet, Tau_idx)*1000., 2) + GetTau(Tau_idDeepTau2017v2p1VSmu, Tau_idx)*1000. + GetTau(Tau_idDeepTau2017v2p1VSe, Tau_idx)")\
                               .Define("tau_DeepTauVsEle_WP", "GetLog2(GetTau(Tau_idDeepTau2017v2p1VSe, Tau_idx) + 1)")\
                               .Define("tau_DeepTauVsMu_WP", "GetLog2(GetTau(Tau_idDeepTau2017v2p1VSmu, Tau_idx) + 1)")\
                               .Define("tau_DeepTauVsJet_WP", "GetLog2(GetTau(Tau_idDeepTau2017v2p1VSjet, Tau_idx) + 1)")\
                               .Define("tau_Zeppenfeld", "Zeppenfeld(tau_eta, leadjet_eta, subleadjet_eta)")\
                               .Define("tau_TightRegion", "Tau_idx[1]")\
                               .Define("tau_LnTRegion", "1 - Tau_idx[1]")\
                               .Define("tau_jetIdx", "GetTau(Tau_jetIdx, Tau_idx)")\
                               .Define("tauleadTk_ptOverTau", "GetTau(Tau_leadTkPtOverTauPt, Tau_idx)")\
                               .Define("tau_DeepTauVsEle_raw", "GetTau(Tau_rawDeepTau2017v2p1VSe, Tau_idx)")\
                               .Define("tau_DeepTauVsMu_raw", "GetTau(Tau_rawDeepTau2017v2p1VSmu, Tau_idx)")\
                               .Define("tau_DeepTauVsJet_raw", "GetTau(Tau_rawDeepTau2017v2p1VSjet, Tau_idx)")

    return df_tauDefinitions

def scale_factors(df):
    #df_SF = df.Define("wzero", "1")
    df_SF = df.Define("lepSF", "GetLeptonSF(Electron_effSF, Electron_idx, Muon_effSF, Muon_idx, GoodLeptonFamily, IsMC)")\
              .Define("lepUp", "abs(GetLeptonSF(Electron_effSF_errUp, Electron_idx, Muon_effSF_errUp, Muon_idx, GoodLeptonFamily, IsMC))")\
              .Define("lepDown", "abs(GetLeptonSF(Electron_effSF_errDown, Electron_idx, Muon_effSF_errDown, Muon_idx, GoodLeptonFamily, IsMC))")\
              .Define("tauSF", "getTauSF(tau_pt, tau_eta, tau_genPartFlav, IsMC, Year)")\
              .Define("tau_vsjet_Down", "tauSF[0][0]")\
              .Define("tau_vsjet_SF", "tauSF[0][1]")\
              .Define("tau_vsjet_Up", "tauSF[0][2]")\
              .Define("tau_vsele_Down", "tauSF[1][0]")\
              .Define("tau_vsele_SF", "tauSF[1][1]")\
              .Define("tau_vsele_Up", "tauSF[1][2]")\
              .Define("tau_vsmu_Down", "tauSF[2][0]")\
              .Define("tau_vsmu_SF", "tauSF[2][1]")\
              .Define("tau_vsmu_Up", "tauSF[2][2]")\
              .Define("btagSFs", "btagcalc(GoodJets_idx, Jet_pt, Jet_eta, Jet_partonFlavour, Jet_btagDeepFlavB, Jet_btagSF_deepjet_M_up, Jet_btagSF_deepjet_M_down, Jet_btagSF_deepjet_M, Jet_btagDeepB, IsMC, Year)")\
              .Define("btagSF", "btagSFs[0]")\
              .Define("btagUp", "btagSFs[1]")\
              .Define("btagDown", "btagSFs[2]")\
              .Define("mistagUp", "btagSFs[3]")\
              .Define("mistagDown", "btagSFs[4]")\

    return df_SF

#HLT_effLumi,PFDown,PFSF,PFUp,lepDown,lepSF,lepUp,lepton_isPrompt,lepton_pdgid,mT_leptau_MET,mT_tau_MET,nBJets,nJets,ptRel_jj,puDown,puSF,puUp,tauDown,tauSF,tauUp,tau_DecayMode,tau_GenMatch,tau_isPrompt,tau_vsele_Down,tau_vsele_SF,tau_vsele_Up,tau_vsjet_Down,tau_vsjet_SF,tau_vsjet_Up,tau_vsmu_Down,tau_vsmu_SF,tau_vsmu_Up,tauleadTk_ptOverTau

def general_definitions(df):
    
    df_definitions = df.Define("MET_pt", "MET_T1Smear_pt_vec[0]")\
                       .Define("MET_phi", "MET_T1Smear_phi_vec[0]")\
                       .Define("m_jj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)")\
                       .Define("deltaPhi_jj", "deltaPhi(leadjet_phi, subleadjet_phi)")\
                       .Define("deltaPhi_taulep", "deltaPhi(tau_phi, lepton_phi)")\
                       .Define("deltaPhi_tauj1", "deltaPhi(tau_phi, leadjet_phi)")\
                       .Define("deltaPhi_tauj2", "deltaPhi(tau_phi, subleadjet_phi)")\
                       .Define("deltaPhi_lepj1", "deltaPhi(lepton_phi, leadjet_phi)")\
                       .Define("deltaPhi_lepj2", "deltaPhi(lepton_phi, subleadjet_phi)")\
                       .Define("deltaEta_jj", "leadjet_eta - subleadjet_eta")\
                       .Define("deltaEta_taulep", "tau_eta - lepton_eta")\
                       .Define("deltaEta_tauj1", "tau_eta - leadjet_eta")\
                       .Define("deltaEta_tauj2", "tau_eta - subleadjet_eta")\
                       .Define("deltaEta_lepj1", "lepton_eta - leadjet_eta")\
                       .Define("deltaEta_lepj2", "lepton_eta - subleadjet_eta")\
                       .Define("deltaTheta_jj", "deltaTheta(leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("deltaTheta_taulep", "deltaTheta(tau_pt,tau_eta, tau_phi, tau_mass, lepton_pt, lepton_eta, lepton_phi, lepton_mass)")\
                       .Define("deltaTheta_tauj1", "deltaTheta(tau_pt, tau_eta,tau_phi, tau_mass, leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass)")\
                       .Define("deltaTheta_tauj2", "deltaTheta(tau_pt, tau_eta, tau_phi, tau_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("ptRel_tauj1", "get_ptrel(tau_pt, tau_eta, tau_phi, tau_mass, leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass)")\
                       .Define("ptRel_tauj2", "get_ptrel(tau_pt, tau_eta, tau_phi, tau_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("ptRel_lepj1", "get_ptrel(lepton_pt, lepton_eta, lepton_phi, lepton_mass, leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass)")\
                       .Define("ptRel_lepj2", "get_ptrel(lepton_pt, lepton_eta, lepton_phi, lepton_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("ptRel_taulep", "get_ptrel(tau_pt, tau_eta, tau_phi, tau_mass, lepton_pt, lepton_eta, lepton_phi, lepton_mass)")\
                       .Define("ptRel_jj", "get_ptrel(leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("event_Zeppenfeld", "0.5*(lepton_Zeppenfeld + tau_Zeppenfeld)")\
                       .Define("lepton_Zeppenfeld_over_deltaEta_jj", "lepton_Zeppenfeld / deltaEta_jj")\
                       .Define("tau_Zeppenfeld_over_deltaEta_jj", "tau_Zeppenfeld / deltaEta_jj")\
                       .Define("event_Zeppenfeld_over_deltaEta_jj", "event_Zeppenfeld / deltaEta_jj")\
                       .Define("mTlepMET", "sqrt(2*lepton_pt*MET_pt*(1-cos(lepton_phi-MET_phi)))")\
                       .Define("m_1T", "M1T(lepton_pt, lepton_eta, lepton_phi, lepton_mass, tau_pt, tau_eta, tau_phi, tau_mass, MET_pt, MET_phi)")\
                       .Define("m_o1", "Mo1(lepton_pt, lepton_eta, lepton_phi, lepton_mass, tau_pt, tau_eta, tau_phi, tau_mass, MET_pt, MET_phi)")\
                       .Define("m_jjtaulep","GetInvMassNoIndex4(leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass, tau_pt, tau_eta, tau_phi, tau_mass, lepton_pt, lepton_eta, lepton_phi, lepton_mass)")\
                       .Define("m_taulep","GetInvMassNoIndex(tau_pt, tau_eta, tau_phi, tau_mass, lepton_pt, lepton_eta, lepton_phi, lepton_mass)")\
                       .Define("mT_lep_MET","sqrt(2*lepton_pt*MET_pt*(1. - cos(lepton_phi-MET_phi)))")\
                       .Define("mT_tau_MET","sqrt(2*tau_pt*MET_pt*(1. - cos(tau_phi-MET_phi)))")\
                       .Define("taujet_relpt", "taujet_RelPt(tau_jetIdx, tau_pt, Jet_pt)")\
                       .Define("taujet_deltaPhi", "taujet_deltaPhi(tau_jetIdx, tau_phi, Jet_phi)")\
                       .Define("nBJets", "CountBJets(Jet_pt, Jet_eta, Jet_btagDeepFlavB)")\
                       .Define("nJets", "nJet")\
                       .Define("PFSF","PrefireWeight")\
                       .Define("PFUp","PrefireWeight_Up")\
                       .Define("PFDown","PrefireWeight_Down")\
                       .Define("puSF","puWeight")\
                       .Define("puDown","puWeightDown")\
                       .Define("puUp","puWeightUp")\
                       .Define("event_RT","(lepton_pt * tau_pt) / (leadjet_pt * subleadjet_pt)")\
                       .Define("m_jjtau","GetInvMassNoIndex3(leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass, tau_pt, tau_eta, tau_phi, tau_mass)")\
                       .Define("pass_charge_selection", "SameCharge(GoodLeptonFamily, Electron_idx, Electron_charge, Muon_idx, Muon_charge, Tau_idx, Tau_charge)")\
                       .Define("pass_b_veto", "BVeto(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx)")\
                       .Define("pass_b_veto_loose", "BVeto_loose(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx)")\
                       .Define("pass_MET_cut", "MET_pt > 50.")\
                       .Define("pass_mjj_cut", "m_jj > 500.")\
                       .Define("SMbdt", "sofie_functor(rdfslot_, sc(m_jj,0), sc(m_jjtaulep,1) , sc(leadjet_pt,2) , sc(subleadjet_pt,3) , sc(nBJets,4) , sc(m_1T,5) , sc(m_o1,6) , sc(nJet,7) , sc(event_Zeppenfeld_over_deltaEta_jj,8) , sc(mT_lep_MET,9) , sc(deltaPhi_jj,10) ,sc(event_RT,11) , sc(tau_DeepTauVsMu_raw,12) , sc(lepton_pfRelIso04,13) , sc(MET_pt,14) , sc(tau_DeepTauVsEle_raw,15) , sc(taujet_relpt,16) , sc(leadjet_DeepFlv_b,17) , sc(m_taulep,18) , sc(ptRel_lepj1,19) , sc(subleadjet_DeepFlv_b,20) , sc(m_jjtau,21) , sc(lepton_pt,22) , sc(deltaEta_jj,23) , sc(ptRel_tauj1,24) , sc(tau_pt,25) , sc(ptRel_tauj2,26) , sc(ptRel_lepj2,27))")\

    return df_definitions

def general_definitions_data(df):
    df_definitions = df.Define("MET_pt", "MET_T1_pt")\
                       .Define("MET_phi", "MET_T1_phi")\
                       .Define("m_jj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)")\
                       .Define("deltaPhi_jj", "deltaPhi(leadjet_phi, subleadjet_phi)")\
                       .Define("deltaPhi_taulep", "deltaPhi(tau_phi, lepton_phi)")\
                       .Define("deltaPhi_tauj1", "deltaPhi(tau_phi, leadjet_phi)")\
                       .Define("deltaPhi_tauj2", "deltaPhi(tau_phi, subleadjet_phi)")\
                       .Define("deltaPhi_lepj1", "deltaPhi(lepton_phi, leadjet_phi)")\
                       .Define("deltaPhi_lepj2", "deltaPhi(lepton_phi, subleadjet_phi)")\
                       .Define("deltaEta_jj", "leadjet_eta - subleadjet_eta")\
                       .Define("deltaEta_taulep", "tau_eta - lepton_eta")\
                       .Define("deltaEta_tauj1", "tau_eta - leadjet_eta")\
                       .Define("deltaEta_tauj2", "tau_eta - subleadjet_eta")\
                       .Define("deltaEta_lepj1", "lepton_eta - leadjet_eta")\
                       .Define("deltaEta_lepj2", "lepton_eta - subleadjet_eta")\
                       .Define("deltaTheta_jj", "deltaTheta(leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("deltaTheta_taulep", "deltaTheta(tau_pt,tau_eta, tau_phi, tau_mass, lepton_pt, lepton_eta, lepton_phi, lepton_mass)")\
                       .Define("deltaTheta_tauj1", "deltaTheta(tau_pt, tau_eta,tau_phi, tau_mass, leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass)")\
                       .Define("deltaTheta_tauj2", "deltaTheta(tau_pt, tau_eta, tau_phi, tau_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("ptRel_tauj1", "get_ptrel(tau_pt, tau_eta, tau_phi, tau_mass, leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass)")\
                       .Define("ptRel_tauj2", "get_ptrel(tau_pt, tau_eta, tau_phi, tau_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("ptRel_lepj1", "get_ptrel(lepton_pt, lepton_eta, lepton_phi, lepton_mass, leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass)")\
                       .Define("ptRel_lepj2", "get_ptrel(lepton_pt, lepton_eta, lepton_phi, lepton_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("ptRel_taulep", "get_ptrel(tau_pt, tau_eta, tau_phi, tau_mass, lepton_pt, lepton_eta, lepton_phi, lepton_mass)")\
                       .Define("ptRel_jj", "get_ptrel(leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass)")\
                       .Define("event_Zeppenfeld", "0.5*(lepton_Zeppenfeld + tau_Zeppenfeld)")\
                       .Define("lepton_Zeppenfeld_over_deltaEta_jj", "lepton_Zeppenfeld / deltaEta_jj")\
                       .Define("tau_Zeppenfeld_over_deltaEta_jj", "tau_Zeppenfeld / deltaEta_jj")\
                       .Define("event_Zeppenfeld_over_deltaEta_jj", "event_Zeppenfeld / deltaEta_jj")\
                       .Define("mTlepMET", "sqrt(2*lepton_pt*MET_T1_pt*(1-cos(lepton_phi-MET_T1_phi)))")\
                       .Define("m_1T", "M1T(lepton_pt, lepton_eta, lepton_phi, lepton_mass, tau_pt, tau_eta, tau_phi, tau_mass, MET_pt, MET_phi)")\
                       .Define("m_o1", "Mo1(lepton_pt, lepton_eta, lepton_phi, lepton_mass, tau_pt, tau_eta, tau_phi, tau_mass, MET_pt, MET_phi)")\
                       .Define("m_jjtaulep","GetInvMassNoIndex4(leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass, tau_pt, tau_eta, tau_phi, tau_mass, lepton_pt, lepton_eta, lepton_phi, lepton_mass)")\
                       .Define("m_taulep","GetInvMassNoIndex(tau_pt, tau_eta, tau_phi, tau_mass, lepton_pt, lepton_eta, lepton_phi, lepton_mass)")\
                       .Define("mT_lep_MET","sqrt(2*lepton_pt*MET_pt*(1. - cos(lepton_phi-MET_phi)))")\
                       .Define("mT_tau_MET","sqrt(2*tau_pt*MET_pt*(1. - cos(tau_phi-MET_phi)))")\
                       .Define("taujet_relpt", "taujet_RelPt(tau_jetIdx, tau_pt, Jet_pt)")\
                       .Define("taujet_deltaPhi", "taujet_deltaPhi(tau_jetIdx, tau_phi, Jet_phi)")\
                       .Define("nBJets", "CountBJets(Jet_pt, Jet_eta, Jet_btagDeepFlavB)")\
                       .Define("nJets", "nJet")\
                       .Define("event_RT","(lepton_pt * tau_pt) / (leadjet_pt * subleadjet_pt)")\
                       .Define("m_jjtau","GetInvMassNoIndex3(leadjet_pt, leadjet_eta, leadjet_phi, leadjet_mass, subleadjet_pt, subleadjet_eta, subleadjet_phi, subleadjet_mass, tau_pt, tau_eta, tau_phi, tau_mass)")\
                       .Define("pass_charge_selection", "SameCharge(GoodLeptonFamily, Electron_idx, Electron_charge, Muon_idx, Muon_charge, Tau_idx, Tau_charge)")\
                       .Define("pass_b_veto", "BVeto(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx)")\
                       .Define("pass_b_veto_loose", "BVeto_loose(Jet_pt, Jet_eta, Jet_btagDeepFlavB, GoodJets_idx)")\
                       .Define("pass_MET_cut", "MET_pt > 50.")\
                       .Define("pass_mjj_cut", "m_jj > 500.")\
                       #.Define("SMbdt", "sofie_functor(rdfslot_, sc(m_jj,0), sc(m_jjtaulep,1) , sc(leadjet_pt,2) , sc(subleadjet_pt,3) , sc(nBJets,4) , sc(m_1T,5) , sc(m_o1,6) , sc(nJet,7) , sc(event_Zeppenfeld_over_deltaEta_jj,8) , sc(mT_lep_MET,9) , sc(deltaPhi_jj,10) ,sc(event_RT,11) , sc(tau_DeepTauVsMu_raw,12) , sc(lepton_pfRelIso04,13) , sc(MET_pt,14) , sc(tau_DeepTauVsEle_raw,15) , sc(taujet_relpt,16) , sc(leadjet_DeepFlv_b,17) , sc(m_taulep,18) , sc(ptRel_lepj1,19) , sc(subleadjet_DeepFlv_b,20) , sc(m_jjtau,21) , sc(lepton_pt,22) , sc(deltaEta_jj,23) , sc(ptRel_tauj1,24) , sc(tau_pt,25) , sc(ptRel_tauj2,26) , sc(ptRel_lepj2,27))")\

    return df_definitions

def book_histos_old(df, region, h):
    h[region] = {} 
    for item in aggregated_samples.items():
        h[region][item[0]] = {}
    h[region]["Fake"] = {}
    for item in aggregated_samples.items():
        filter_string = ""
        for s in item[1]:
            #print(s.name)
            #print(sampleDict[s.name])
            if filter_string == "":
                filter_string += "Sample == {}".format(sampleDict[s.name])
            else:
                filter_string += " || Sample == {}".format(sampleDict[s.name])
        #print(filter_string)
        for v in variables[region]:
            df_sample = df.Filter(filter_string)
            if item[0] == "Data":
                h[region]["Data"][v._name] = {}
                h[region]["Fake"][v._name] = {}
                h[region]['Data'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                   .Filter("event_SFFake_vsjet4 == 0.", "Both lepton and tau are prompt")\
                                                   .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                h[region]['Fake'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                    .Filter("event_SFFake_vsjet4 > 0.", "At least one among lepton and tau is fake")\
                                                    .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet4")
                h[region]['Data'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                   .Filter("event_SFFake_vsjet2 == 0.", "Both lepton and tau are prompt")\
                                                   .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                h[region]['Fake'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                   .Filter("event_SFFake_vsjet2 > 0.", "At least one among lepton and tau is fake")\
                                                   .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet2")
            else:
                h[region][item[0]][v._name] = {}
                h[region][item[0]][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0").Filter("lepton_TightRegion==1").Filter("tau_TightRegion==1").Filter("(lepton_genPartFlav==1 || lepton_genPartFlav==15) && tau_genPartFlav==5", "prompt!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")
                h[region][item[0]][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1").Filter("lepton_TightRegion==1").Filter("tau_TightRegion==1").Filter("(lepton_genPartFlav==1 || lepton_genPartFlav==15) && tau_genPartFlav==5", "prompt!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")

def book_histos(df, region, h, to_plot = []):
    h[region] = {} 
    for item in aggregated_samples.items():
        h[region][item[0]] = {}
    h[region]["Fake"] = {}
    for item in aggregated_samples.items():
        if item[0] in to_plot:
            filter_string = ""
            for s in item[1]:
                #print(s.name)
                #print(sampleDict[s.name])
                if filter_string == "":
                    filter_string += "Sample == {}".format(sampleDict[s.name])
                else:
                    filter_string += " || Sample == {}".format(sampleDict[s.name])
            #print(filter_string)
            for v in variables[region]:
                if not v._name in variables_to_plot:
                    continue
                df_sample = df.Filter(filter_string)
                if item[0] == "Data":
                    h[region]["Data"][v._name] = {}
                    h[region]["Fake"][v._name] = {}
                    h[region]['Data'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                       .Filter("event_SFFake_vsjet4 == 0.", "Both lepton and tau are prompt")\
                                                       .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                    h[region]['Fake'][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0")\
                                                        .Filter("event_SFFake_vsjet4 > 0.", "At least one among lepton and tau is fake")\
                                                        .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet4")
                    h[region]['Data'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                       .Filter("event_SFFake_vsjet2 == 0.", "Both lepton and tau are prompt")\
                                                       .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name)
                    h[region]['Fake'][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1")\
                                                       .Filter("event_SFFake_vsjet2 > 0.", "At least one among lepton and tau is fake")\
                                                       .Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "event_SFFake_vsjet2")
                else:
                    h[region][item[0]][v._name] = {}
                    h[region][item[0]][v._name]['etau'] = df_sample.Filter("GoodLeptonFamily==0", "electron!").Filter("lepton_TightRegion==1", "tight electron!").Filter("tau_TightRegion==1", "tight tau!").Filter("(lepton_genPartFlav==1|| lepton_genPartFlav==15) && tau_genPartFlav==5", "prompt electron and tau!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")
                    h[region][item[0]][v._name]['mutau'] = df_sample.Filter("GoodLeptonFamily==1", "muon!").Filter("lepton_TightRegion==1", "tight_muon!").Filter("tau_TightRegion==1", "tight tau!").Filter("(lepton_genPartFlav==1|| lepton_genPartFlav==15) && tau_genPartFlav==5", "prompt muon and tau!").Histo1D(("h_" + v._name + "_" + region, v._title , v._nbins, v._bins), v._name, "wzero")
                
def produce_tau_SF(df):
    df_SF = df.Define("FESs", "getFES(Tau_eta, Tau_decayMode, Tau_genPartFlav, IsMC, Year)")\
               .Define("FESDown", "getFlattenedMatrixColumn(FESs, 3, 0)")\
               .Define("FESSF", "getFlattenedMatrixColumn(FESs, 3, 1)")\
               .Define("FESUp", "getFlattenedMatrixColumn(FESs, 3, 2)")\
               .Define("TESs", "getTES(Tau_pt, Tau_decayMode,Tau_genPartFlav, IsMC, Year)")\
               .Define("TESDown", "getFlattenedMatrixColumn(TESs, 3, 0)")\
               .Define("TESSF", "getFlattenedMatrixColumn(TESs, 3, 1)")\
               .Define("TESUp", "getFlattenedMatrixColumn(TESs, 3, 2)")
    return df_SF
                
def jet_tau_redefines(df):
    df_result = df.Redefine("Jet_pt", "Jet_pt_nom")\
                  .Redefine("Jet_mass", "Jet_mass_nom")
    return df_result


def energetic_variations(df):
    df_sys = df.Vary("TESSF", "RVec<RVec<float>>{TESDown, TESUp}", variationTags=["down", "up"], variationName="TES")\
               .Vary("FESSF", "RVec<RVec<float>>{FESDown, FESUp}", variationTags=["down", "up"], variationName="FES")\
               .Vary(["Jet_pt", "Jet_mass", "MET_T1Smear_pt_vec", "MET_T1Smear_phi_vec"], "RVec<RVec<RVec<float>>>{{Jet_pt_jerDown, Jet_pt_jerUp}, {Jet_mass_jerDown, Jet_mass_jerUp}, {MET_T1Smear_pt_jerDown_vec, MET_T1Smear_pt_jerUp_vec}, {MET_T1Smear_phi_jerDown_vec, MET_T1Smear_phi_jerUp_vec}}", variationTags=["down", "up"], variationName="jer")\
               .Vary(["Jet_pt", "Jet_mass", "MET_T1Smear_pt_vec", "MET_T1Smear_phi_vec"], "RVec<RVec<RVec<float>>>{{Jet_pt_jesTotalDown, Jet_pt_jesTotalUp}, {Jet_mass_jesTotalDown, Jet_mass_jesTotalUp}, {MET_T1Smear_pt_jesTotalDown_vec, MET_T1Smear_pt_jesTotalUp_vec}, {MET_T1Smear_phi_jesTotalDown_vec, MET_T1Smear_phi_jesTotalUp_vec}}", variationTags=["down", "up"], variationName="jesTotal")\
               #.Define("MET_T1Smear_pt_vec", "RVec<float>{ (float) MET_T1Smear_pt}").Define("MET_T1Smear_phi_vec", "RVec<float>{ (float) MET_T1Smear_phi}")\
               #.Define("MET_T1Smear_pt_jerDown_vec", "RVec<float>{ (float) MET_T1Smear_pt_jerDown}").Define("MET_T1Smear_phi_jerDown_vec", "RVec<float>{ (float) MET_T1Smear_phi_jerDown}")\
               #.Define("MET_T1Smear_pt_jerUp_vec", "RVec<float>{ (float) MET_T1Smear_pt_jerUp}").Define("MET_T1Smear_phi_jerUp_vec", "RVec<float>{ (float) MET_T1Smear_phi_jerUp}")\
               #.Define("MET_T1Smear_pt_jesTotalDown_vec", "RVec<float>{ (float) MET_T1Smear_pt_jesTotalDown}").Define("MET_T1Smear_phi_jesTotalDown_vec", "RVec<float>{ (float) MET_T1Smear_phi_jesTotalDown}")\
               #.Define("MET_T1Smear_pt_jesTotalUp_vec", "RVec<float>{ (float) MET_T1Smear_pt_jesTotalUp}").Define("MET_T1Smear_phi_jesTotalUp_vec", "RVec<float>{ (float) MET_T1Smear_phi_jesTotalUp}")\

    return df_sys

def SF_variations(df):
    df_sys = df.Vary("PrefireWeight", "RVec<float>{PrefireWeight_Down, PrefireWeight_Up}", variationTags=["down", "up"], variationName="PF")\
               .Vary("puWeight", "RVec<float>{puWeightDown, puWeightUp}", variationTags=["down", "up"], variationName="pu")\
               .Vary("btagSF", "RVec<float>{btagDown, btagUp}", variationTags=["down", "up"], variationName="btag")\
               .Vary("lepSF", "RVec<float>{lepDown, lepUp}", variationTags=["down", "up"], variationName="lep")\
               .Vary("tau_vsjet_SF", "RVec<float>{tau_vsjet_Down, tau_vsjet_Up}", variationTags=["down", "up"], variationName="tau_vsjet")\
               .Vary("tau_vsele_SF", "RVec<float>{tau_vsele_Down, tau_vsele_Up}", variationTags=["down", "up"], variationName="tau_vsele")\
               .Vary("tau_vsmu_SF", "RVec<float>{tau_vsmu_Down, tau_vsmu_Up}", variationTags=["down", "up"], variationName="tau_vsmu")\
               .Vary("pdf_totalSF", "RVec<float>{pdf_totalDown, pdf_totalUp}", variationTags=["down", "up"], variationName="pdf_total")\
               .Vary("QCDScaleSF", "RVec<float>{QCDScaleDown, QCDScaleUp}", variationTags=["down", "up"], variationName="QCDScale")\
               .Vary("ISRSF", "RVec<float>{ISRDown, ISRUp}", variationTags=["down", "up"], variationName="ISR")\
               .Vary("FSRSF", "RVec<float>{FSRDown, FSRUp}", variationTags=["down", "up"], variationName="FSR")\
    
    return df_sys

# Columns to save in the snapshot

In [9]:
MC_columns = ["Sample","FSRDown", "FSRSF", "FSRUp", "HLT_effLumi", "ISRDown", "ISRSF", "ISRUp", "MET_phi", "MET_pt", "PFDown", "PFSF", "PFUp", "QCDScaleDown", "QCDScaleSF", "QCDScaleUp", "btagDown", "btagSF", "btagUp", "deltaEta_jj", "deltaEta_lepj1", "deltaEta_lepj2", "deltaEta_tauj1", "deltaEta_tauj2", "deltaEta_taulep", "deltaPhi_jj", "deltaPhi_lepj1", "deltaPhi_lepj2", "deltaPhi_tauj1", "deltaPhi_tauj2", "deltaPhi_taulep", "event_RT",  "event_Zeppenfeld", "event_Zeppenfeld_over_deltaEta_jj", "leadjet_DeepFlv_b", "leadjet_eta", "leadjet_mass", "leadjet_phi", "leadjet_pt", "lepDown", "lepSF", "lepUp", "lepton_LnTRegion", "lepton_TightRegion", "lepton_Zeppenfeld", "lepton_Zeppenfeld_over_deltaEta_jj", "lepton_eta", "lepton_isPrompt", "lepton_mass", "lepton_pdgid", "lepton_pfRelIso04", "lepton_phi", "lepton_pt", "mT_lep_MET", "m_1T", "m_jj", "m_jjtau", "m_jjtaulep", "m_o1", "m_taulep", #"mistagUp", 
              "nBJets", "nJets", "pass_MET_cut", "pass_b_veto", "pass_b_veto_loose", "pass_charge_selection", "pass_mjj_cut", "pdf_totalDown", "pdf_totalSF", "pdf_totalUp", "ptRel_jj", "ptRel_lepj1", "ptRel_lepj2", "ptRel_tauj1", "ptRel_tauj2", "ptRel_taulep", "puDown", "puSF", "puUp", "subleadjet_DeepFlv_b", "subleadjet_eta", "subleadjet_mass", "subleadjet_phi", "subleadjet_pt", "tau_DecayMode", "tau_DeepTauVsEle_WP", "tau_DeepTauVsEle_raw", "tau_DeepTauVsJet_WP", "tau_DeepTauVsJet_raw", "tau_DeepTauVsMu_WP", "tau_DeepTauVsMu_raw", "tau_DeepTau_WP", "tau_LnTRegion", "tau_TightRegion", "tau_Zeppenfeld", "tau_Zeppenfeld_over_deltaEta_jj", "tau_eta", "tau_isPrompt", "tau_isolation", "tau_mass", "tau_phi", "tau_pt", "tau_vsele_Down", "tau_vsele_SF", "tau_vsele_Up", "tau_vsjet_Down", "tau_vsjet_SF", "tau_vsjet_Up", "tau_vsmu_Down", "tau_vsmu_SF", "tau_vsmu_Up", "tauleadTk_ptOverTau", "w_nominal", "SMbdt"]
Data_columns = ["Sample", "HLT_effLumi", "MET_phi", "MET_pt", "deltaEta_jj", "deltaEta_lepj1", "deltaEta_lepj2", "deltaEta_tauj1", "deltaEta_tauj2", "deltaEta_taulep", "deltaPhi_jj", "deltaPhi_lepj1", "deltaPhi_lepj2", "deltaPhi_tauj1", "deltaPhi_tauj2", "deltaPhi_taulep", "event_RT", "event_SFFake_vsjet2", "event_SFFake_vsjet4", "event_Zeppenfeld", "event_Zeppenfeld_over_deltaEta_jj", "leadjet_DeepFlv_b", "leadjet_eta", "leadjet_mass", "leadjet_phi", "leadjet_pt", "lepDown", "lepSF", "lepUp", "lepton_LnTRegion", "lepton_SFFake_vsjet2", "lepton_SFFake_vsjet4", "lepton_TightRegion", "lepton_Zeppenfeld", "lepton_Zeppenfeld_over_deltaEta_jj", "lepton_eta", "lepton_isPrompt", "lepton_mass", "lepton_pdgid", "lepton_pfRelIso04", "lepton_phi", "lepton_pt", "mT_lep_MET", "m_1T", "m_jj", "m_jjtau", "m_jjtaulep", "m_o1", "m_taulep", #"mistagUp", 
                "nBJets", "nJets", "pass_MET_cut", "pass_b_veto", "pass_b_veto_loose", "pass_charge_selection", "pass_mjj_cut", "pdf_totalDown", "pdf_totalSF", "pdf_totalUp", "ptRel_jj", "ptRel_lepj1", "ptRel_lepj2", "ptRel_tauj1", "ptRel_tauj2", "ptRel_taulep", "subleadjet_DeepFlv_b", "subleadjet_eta", "subleadjet_mass", "subleadjet_phi", "subleadjet_pt", "tau_DecayMode", "tau_DeepTauVsEle_WP", "tau_DeepTauVsEle_raw", "tau_DeepTauVsJet_WP", "tau_DeepTauVsJet_raw", "tau_DeepTauVsMu_WP", "tau_DeepTauVsMu_raw", "tau_DeepTau_WP", "tau_LnTRegion", "tau_SFFake_vsjet2", "tau_SFFake_vsjet4", "tau_TightRegion", "tau_Zeppenfeld", "tau_Zeppenfeld_over_deltaEta_jj", "tau_eta", "tau_isPrompt", "tau_isolation", "tau_mass", "tau_phi", "tau_pt", "tau_vsele_Down", "tau_vsele_SF", "tau_vsele_Up", "tau_vsjet_Down", "tau_vsjet_SF", "tau_vsjet_Up", "tau_vsmu_Down", "tau_vsmu_SF", "tau_vsmu_Up", "tauleadTk_ptOverTau", "w_nominal", "SMbdt"]

# Define samples to be analyzed

In [10]:
sampleDict = {'ZZtoLep_UL2016APV': 0,'ZZTo2L2Nu_UL2016APV': 1,'ZZTo4L_UL2016APV': 2,'GluGluToContinToZZTo4e_UL2016APV': 3,'GluGluToContinToZZTo2e2mu_UL2016APV': 4,'GluGluToContinToZZTo2e2tau_UL2016APV': 5,'GluGluToContinToZZTo2mu2nu_UL2016APV': 6,'GluGluToContinToZZTo4mu_UL2016APV': 7,'GluGluToContinToZZTo2mu2tau_UL2016APV': 8,'GluGluToContinToZZTo2tau2nu_UL2016APV': 9,'GluGluToContinToZZTo4tau_UL2016APV': 10,'GluGluToContinToZZTo2e2nu_UL2016APV': 11,'TT_UL2016APV': 12,'TT_SemiLep_UL2016APV': 13,'TT_Had_UL2016APV': 14,'TTTo2L2Nu_UL2016APV': 15,'TT_beff_UL2016APV': 16,'TVX_UL2016APV': 17,'TTGJets_UL2016APV': 18,'TTZToQQ_UL2016APV': 19,'TTZToLLNuNu_UL2016APV': 20,'TTWJetsToQQ_UL2016APV': 21,'TTWJetsToLNu_UL2016APV': 22,'tZq_ll_4f_UL2016APV': 23,'VG_UL2016APV': 24,'ZG_UL2016APV': 25,'WG_UL2016APV': 26,'WrongSign_UL2016APV': 27,'WWto2L2Nu_UL2016APV': 28,'GluGluToWWToENEN_UL2016APV': 29,'GluGluToWWToENMN_UL2016APV': 30,'GluGluToWWToENTN_UL2016APV': 31,'GluGluToWWToMNEN_UL2016APV': 32,'GluGluToWWToMNMN_UL2016APV': 33,'GluGluToWWToMNTN_UL2016APV': 34,'GluGluToWWToTNEN_UL2016APV': 35,'GluGluToWWToTNMN_UL2016APV': 36,'GluGluToWWToTNTN_UL2016APV': 37,'ST_tW_top_UL2016APV': 38,'ST_tW_antitop_UL2016APV': 39,'GluGluHToWWTo2L2Nu_UL2016APV': 40,'GluGluHToWWToLNuQQ_UL2016APV': 41,'GluGluHToZZTo4L_UL2016APV': 42,'GluGluHToTauTau_UL2016APV': 43,'VBFHToWWTo2L2Nu_UL2016APV': 44,'VBFHToTauTau_UL2016APV': 45,'ttHToNonbb_UL2016APV': 46,'VHToNonbb_UL2016APV': 47,'Triboson_UL2016APV': 48,'WWTo2L2Nu_DoubleScattering_UL2016': 49,'WWW_4F_UL2016APV': 50,'WWZ_4F_UL2016APV': 51,'WZZ_UL2016APV': 52,'ZZZ_UL2016APV': 53,'WWG_UL2016APV': 54,'WJets_UL2016APV': 55,'WJetsHT70to100_UL2016APV': 56,'WJetsHT100to200_UL2016APV': 57,'WJetsHT200to400_UL2016APV': 58,'WJetsHT400to600_UL2016APV': 59,'WJetsHT600to800_UL2016APV': 60,'WJetsHT800to1200_UL2016APV': 61,'WJetsHT1200to2500_UL2016APV': 62,'WJetsHT2500toInf_UL2016APV': 63,'WZ_UL2016APV': 64,'DYJetsToLL_UL2016APV': 65,'DYJetsToLL_FxFx_UL2016APV': 65,'DYJetsToLL_M10to50_UL2016APV': 66,'DYJetsToLL_M50_UL2016APV': 67,'DYJetsToLL_M50_FxFx_UL2016APV': 67,'DYJetsToLL_M50_UL2016APV_ext': 67,'WpWpJJ_EWK_UL2016APV': 68,'WpWpJJ_QCD_UL2016APV': 69,'VBS_SSWW_SM_UL2016APV': 70,'VBS_SSWW_LL_SM_UL2016APV': 71,'VBS_SSWW_TL_SM_UL2016APV': 72,'VBS_SSWW_TT_SM_UL2016APV': 73,'VBS_SSWW_cW_UL2016APV': 74,'VBS_SSWW_cW_SM_UL2016APV': 75,'VBS_SSWW_cW_BSM_UL2016APV': 76,'VBS_SSWW_cW_INT_UL2016APV': 77,'VBS_SSWW_cHW_UL2016APV': 78,'VBS_SSWW_cHW_SM_UL2016APV': 79,'VBS_SSWW_cHW_BSM_UL2016APV': 80,'VBS_SSWW_cHW_INT_UL2016APV': 81,'VBS_SSWW_cW_cHW_UL2016APV': 82,'VBS_SSWW_DIM6_UL2016APV': 83,'VBS_SSWW_DIM6_SM_UL2016APV': 84,'ZZtoLep_UL2016': 85,'ZZTo2L2Nu_UL2016': 86,'ZZTo4L_UL2016': 87,'GluGluToContinToZZTo4e_UL2016': 88,'GluGluToContinToZZTo2e2mu_UL2016': 89,'GluGluToContinToZZTo2e2tau_UL2016': 90,'GluGluToContinToZZTo2mu2nu_UL2016': 91,'GluGluToContinToZZTo4mu_UL2016': 92,'GluGluToContinToZZTo2mu2tau_UL2016': 93,'GluGluToContinToZZTo2tau2nu_UL2016': 94,'GluGluToContinToZZTo4tau_UL2016': 95,'GluGluToContinToZZTo2e2nu_UL2016': 96,'TT_UL2016': 97,'TT_SemiLep_UL2016': 98,'TT_Had_UL2016': 99,'TTTo2L2Nu_UL2016': 100,'TT_beff_UL2016': 101,'TVX_UL2016': 102,'TTGJets_UL2016': 103,'TTZToQQ_UL2016': 104,'TTZToLLNuNu_UL2016': 105,'TTWJetsToQQ_UL2016': 106,'TTWJetsToLNu_UL2016': 107,'tZq_ll_4f_UL2016': 108,'VG_UL2016': 109,'ZG_UL2016': 110,'WG_UL2016': 111,'WrongSign_UL2016': 112,'WWto2L2Nu_UL2016': 113,'GluGluToWWToENEN_UL2016': 114,'GluGluToWWToENMN_UL2016': 115,'GluGluToWWToENTN_UL2016': 116,'GluGluToWWToMNEN_UL2016': 117,'GluGluToWWToMNMN_UL2016': 118,'GluGluToWWToMNTN_UL2016': 119,'GluGluToWWToTNEN_UL2016': 120,'GluGluToWWToTNMN_UL2016': 121,'GluGluToWWToTNTN_UL2016': 122,'ST_tW_top_UL2016': 123,'ST_tW_antitop_UL2016': 124,'GluGluHToWWTo2L2Nu_UL2016': 125,'GluGluHToWWToLNuQQ_UL2016': 126,'GluGluHToZZTo4L_UL2016': 127,'GluGluHToTauTau_UL2016': 128,'VBFHToWWTo2L2Nu_UL2016': 129,'VBFHToTauTau_UL2016': 130,'ttHToNonbb_UL2016': 131,'VHToNonbb_UL2016': 132,'Triboson_UL2016': 133,'WWW_4F_UL2016': 134,'WWZ_4F_UL2016': 135,'WZZ_UL2016': 136,'ZZZ_UL2016': 137,'WWG_UL2016': 138,'WJets_UL2016': 139,'WJetsHT70to100_UL2016': 140,'WJetsHT100to200_UL2016': 141,'WJetsHT200to400_UL2016': 142,'WJetsHT400to600_UL2016': 143,'WJetsHT600to800_UL2016': 144,'WJetsHT800to1200_UL2016': 145,'WJetsHT1200to2500_UL2016': 146,'WJetsHT2500toInf_UL2016': 147,'WZ_UL2016': 148,'DYJetsToLL_UL2016': 149,'DYJetsToLL_FxFx_UL2016': 149,'DYJetsToLL_M10to50_UL2016': 150,'DYJetsToLL_M50_UL2016': 151,'DYJetsToLL_M50_FxFx_UL2016': 151,'DYJetsToLL_M50_UL2016_ext': 151,'WpWpJJ_EWK_UL2016': 152,'WpWpJJ_QCD_UL2016': 153,'VBS_SSWW_SM_UL2016': 154,'VBS_SSWW_LL_SM_UL2016': 155,'VBS_SSWW_TL_SM_UL2016': 156,'VBS_SSWW_TT_SM_UL2016': 157,'VBS_SSWW_cW_UL2016': 158,'VBS_SSWW_cW_SM_UL2016': 159,'VBS_SSWW_cW_BSM_UL2016': 160,'VBS_SSWW_cW_INT_UL2016': 161,'VBS_SSWW_cHW_UL2016': 162,'VBS_SSWW_cHW_SM_UL2016': 163,'VBS_SSWW_cHW_BSM_UL2016': 164,'VBS_SSWW_cHW_INT_UL2016': 165,'VBS_SSWW_cW_cHW_UL2016': 166,'VBS_SSWW_DIM6_UL2016': 167,'VBS_SSWW_DIM6_SM_UL2016': 168,'ZZtoLep_UL2017': 169,'ZZTo2L2Nu_UL2017': 170,'ZZTo4L_UL2017': 171,'GluGluToContinToZZTo4e_UL2017': 172,'GluGluToContinToZZTo2e2mu_UL2017': 173,'GluGluToContinToZZTo2e2tau_UL2017': 174,'GluGluToContinToZZTo2mu2nu_UL2017': 175,'GluGluToContinToZZTo4mu_UL2017': 176,'GluGluToContinToZZTo2mu2tau_UL2017': 177,'GluGluToContinToZZTo2tau2nu_UL2017': 178,'GluGluToContinToZZTo4tau_UL2017': 179,'GluGluToContinToZZTo2e2nu_UL2017': 180,'TT_UL2017': 181,'TT_SemiLep_UL2017': 182,'TT_Had_UL2017': 183,'TTTo2L2Nu_UL2017': 184,'TT_beff_UL2017': 185,'TVX_UL2017': 186,'TTGJets_UL2017': 187,'TTZToQQ_UL2017': 188,'TTZToLLNuNu_UL2017': 189,'TTWJetsToQQ_UL2017': 190,'TTWJetsToLNu_UL2017': 191,'tZq_ll_4f_UL2017': 192,'VG_UL2017': 193,'ZG_UL2017': 194,'WG_UL2017': 195,'WrongSign_UL2017': 196,'WWto2L2Nu_UL2017': 197,'GluGluToWWToENEN_UL2017': 198,'GluGluToWWToENMN_UL2017': 199,'GluGluToWWToENTN_UL2017': 200,'GluGluToWWToMNEN_UL2017': 201,'GluGluToWWToMNMN_UL2017': 202,'GluGluToWWToMNTN_UL2017': 203,'GluGluToWWToTNEN_UL2017': 204,'GluGluToWWToTNMN_UL2017': 205,'GluGluToWWToTNTN_UL2017': 206,'ST_tW_top_UL2017': 207,'ST_tW_antitop_UL2017': 208,'GluGluHToWWTo2L2Nu_UL2017': 209,'GluGluHToWWToLNuQQ_UL2017': 210,'GluGluHToZZTo4L_UL2017': 211,'GluGluHToTauTau_UL2017': 212,'VBFHToWWTo2L2Nu_UL2017': 213,'VBFHToTauTau_UL2017': 214,'ttHToNonbb_UL2017': 215,'VHToNonbb_UL2017': 216,'Triboson_UL2017': 217,'WWTo2L2Nu_DoubleScattering_UL2017': 218,'WWW_4F_UL2017': 219,'WWZ_4F_UL2017': 220,'WZZ_UL2017': 221,'ZZZ_UL2017': 222,'WWG_UL2017': 223,'WJets_UL2017': 224,'WJetsHT70to100_UL2017': 225,'WJetsHT100to200_UL2017': 226,'WJetsHT200to400_UL2017': 227,'WJetsHT400to600_UL2017': 228,'WJetsHT600to800_UL2017': 229,'WJetsHT800to1200_UL2017': 230,'WJetsHT1200to2500_UL2017': 231,'WJetsHT2500toInf_UL2017': 232,'WZ_UL2017': 233,'DYJetsToLL_UL2017': 234,'DYJetsToLL_FxFx_UL2017': 234,'DYJetsToLL_M10to50_UL2017': 235,'DYJetsToLL_M50_UL2017': 236,'DYJetsToLL_M50_FxFx_UL2017': 236,'DYJetsToLL_M50_UL2017_ext': 236,'WpWpJJ_EWK_UL2017': 237,'WpWpJJ_QCD_UL2017': 238,'VBS_SSWW_SM_UL2017': 239,'VBS_SSWW_LL_SM_UL2017': 240,'VBS_SSWW_TL_SM_UL2017': 241,'VBS_SSWW_TT_SM_UL2017': 242,'VBS_SSWW_cW_UL2017': 243,'VBS_SSWW_cW_SM_UL2017': 244,'VBS_SSWW_cW_BSM_UL2017': 245,'VBS_SSWW_cW_INT_UL2017': 246,'VBS_SSWW_cHW_UL2017': 247,'VBS_SSWW_cHW_SM_UL2017': 248,'VBS_SSWW_cHW_BSM_UL2017': 249,'VBS_SSWW_cHW_INT_UL2017': 250,'VBS_SSWW_cW_cHW_UL2017': 251,'VBS_SSWW_DIM6_UL2017': 252,'VBS_SSWW_DIM6_SM_UL2017': 253,'ZZtoLep_UL2018': 254,'ZZTo2L2Nu_UL2018': 255,'ZZTo4L_UL2018': 256,'GluGluToContinToZZTo4e_UL2018': 257,'GluGluToContinToZZTo2e2mu_UL2018': 258,'GluGluToContinToZZTo2e2tau_UL2018': 259,'GluGluToContinToZZTo2mu2nu_UL2018': 260,'GluGluToContinToZZTo4mu_UL2018': 261,'GluGluToContinToZZTo2mu2tau_UL2018': 262,'GluGluToContinToZZTo2tau2nu_UL2018': 263,'GluGluToContinToZZTo4tau_UL2018': 264,'GluGluToContinToZZTo2e2nu_UL2018': 265,'TT_UL2018': 266,'TT_SemiLep_UL2018': 267,'TT_Had_UL2018': 268,'TTTo2L2Nu_UL2018': 269,'TT_beff_UL2018': 270,'TVX_UL2018': 271,'TTGJets_UL2018': 272,'TTZToQQ_UL2018': 273,'TTZToLLNuNu_UL2018': 274,'TTWJetsToQQ_UL2018': 275,'TTWJetsToLNu_UL2018': 276,'tZq_ll_4f_UL2018': 277,'VG_UL2018': 278,'ZG_UL2018': 279,'WG_UL2018': 280,'WrongSign_UL2018': 281,'WWto2L2Nu_UL2018': 282,'GluGluToWWToENEN_UL2018': 283,'GluGluToWWToENMN_UL2018': 284,'GluGluToWWToENTN_UL2018': 285,'GluGluToWWToMNEN_UL2018': 286,'GluGluToWWToMNMN_UL2018': 287,'GluGluToWWToMNTN_UL2018': 288,'GluGluToWWToTNEN_UL2018': 289,'GluGluToWWToTNMN_UL2018': 290,'GluGluToWWToTNTN_UL2018': 291,'ST_tW_top_UL2018': 292,'ST_tW_antitop_UL2018': 293,'GluGluHToWWTo2L2Nu_UL2018': 294,'GluGluHToWWToLNuQQ_UL2018': 295,'GluGluHToZZTo4L_UL2018': 296,'GluGluHToTauTau_UL2018': 297,'VBFHToWWTo2L2Nu_UL2018': 298,'VBFHToTauTau_UL2018': 299,'ttHToNonbb_UL2018': 300,'VHToNonbb_UL2018': 301,'Triboson_UL2018': 302,'WWTo2L2Nu_DoubleScattering_UL2018': 303,'WWW_4F_UL2018': 304,'WWZ_4F_UL2018': 305,'WZZ_UL2018': 306,'ZZZ_UL2018': 307,'WWG_UL2018': 308,'WJets_UL2018': 309,'WJetsHT70to100_UL2018': 310,'WJetsHT100to200_UL2018': 311,'WJetsHT200to400_UL2018': 312,'WJetsHT400to600_UL2018': 313,'WJetsHT600to800_UL2018': 314,'WJetsHT800to1200_UL2018': 315,'WJetsHT1200to2500_UL2018': 316,'WJetsHT2500toInf_UL2018': 317,'WZ_UL2018': 318,'DYJetsToLL_UL2018': 319,'DYJetsToLL_FxFx_UL2018': 319,'DYJetsToLL_M10to50_UL2018': 320,'DYJetsToLL_M50_UL2018': 321,'DYJetsToLL_M50_FxFx_UL2018': 321,'DYJetsToLL_M50_UL2018_ext': 321,'WpWpJJ_EWK_UL2018': 322,'WpWpJJ_QCD_UL2018': 323,'VBS_SSWW_SM_UL2018': 324,'VBS_SSWW_LL_SM_UL2018': 325,'VBS_SSWW_TL_SM_UL2018': 326,'VBS_SSWW_TT_SM_UL2018': 327,'VBS_SSWW_cW_UL2018': 328,'VBS_SSWW_cW_BSM_UL2018': 329,'VBS_SSWW_cW_SM_UL2018': 330,'VBS_SSWW_cW_INT_UL2018': 331,'VBS_SSWW_cHW_UL2018': 332,'VBS_SSWW_cHW_SM_UL2018': 333,'VBS_SSWW_cHW_BSM_UL2018': 334,'VBS_SSWW_cHW_INT_UL2018': 335,'VBS_SSWW_cW_cHW_UL2018': 336,'VBS_SSWW_DIM6_UL2018': 337,'VBS_SSWW_DIM6_SM_UL2018': 338,'DataMu_UL2016APV': 339,'DataMuB1_UL2016APV': 340,'DataMuB2_UL2016APV': 341,'DataMuC_UL2016APV': 342,'DataMuD_UL2016APV': 343,'DataMuE_UL2016APV': 344,'DataMuF_UL2016APV': 345,'DataMu_UL2016': 346,'DataMuF_UL2016': 347,'DataMuG_UL2016': 348,'DataMuH_UL2016': 349,'DataMu_UL2017': 350,'DataMuB_UL2017': 351,'DataMuC_UL2017': 352,'DataMuD_UL2017': 353,'DataMuE_UL2017': 354,'DataMuF_UL2017': 355,'DataMu_UL2018': 356,'DataMuA_UL2018': 357,'DataMuB_UL2018': 358,'DataMuC_UL2018': 359,'DataMuD_UL2018': 360,'DataEle_UL2016APV': 361,'DataEleB1_UL2016APV': 362,'DataEleB2_UL2016APV': 363,'DataEleC_UL2016APV': 364,'DataEleD_UL2016APV': 365,'DataEleE_UL2016APV': 366,'DataEleF_UL2016APV': 367,'DataEle_UL2016': 368,'DataEleF_UL2016': 369,'DataEleG_UL2016': 370,'DataEleH_UL2016': 371,'DataEle_UL2017': 372,'DataEleB_UL2017': 373,'DataEleC_UL2017': 374,'DataEleD_UL2017': 375,'DataEleE_UL2017': 376,'DataEleF_UL2017': 377,'DataEle_UL2018': 378,'DataEleA_UL2018': 379,'DataEleB_UL2018': 380,'DataEleC_UL2018': 381,'DataEleD_UL2018': 382,'DataHT_UL2016APV': 383,'DataHTB1_UL2016APV': 384,'DataHTB2_UL2016APV': 385,'DataHTC_UL2016APV': 386,'DataHTD_UL2016APV': 387,'DataHTE_UL2016APV': 388,'DataHTF_UL2016APV': 389,'DataHT_UL2016': 390,'DataHTF_UL2016': 391,'DataHTG_UL2016': 392,'DataHTH_UL2016': 393,'DataHT_UL2017': 394,'DataHTB_UL2017': 395,'DataHTC_UL2017': 396,'DataHTD_UL2017': 397,'DataHTE_UL2017': 398,'DataHTF_UL2017': 399,'DataHT_UL2018': 400,'DataHTA_UL2018': 401,'DataHTB_UL2018': 402,'DataHTC_UL2018': 403,'DataHTD_UL2018': 404,'SampleHTFake_UL2016APV': 405,'SampleHTFake_UL2016': 406,'SampleHTFake_UL2017': 407,'SampleHTFake_UL2018': 408,}

aggregated_samples_UL2016 = {
    'VG':  [ZG_UL2016, WG_UL2016],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2016],   
    'TVX': [TTGJets_UL2016, TTZToQQ_UL2016, TTZToLLNuNu_UL2016, TTWJetsToQQ_UL2016, TTWJetsToLNu_UL2016, tZq_ll_4f_UL2016], 
    'Triboson': [WWW_4F_UL2016, WWZ_4F_UL2016, WZZ_UL2016, ZZZ_UL2016, WWG_UL2016], 
    'TTTo2L2Nu': [TTTo2L2Nu_UL2016], 
    'WZ': [WZ_UL2016], 
    'DYJetsToLL_FxFx': [DYJetsToLL_M50_FxFx_UL2016],
    'WrongSign': [WWto2L2Nu_UL2016, GluGluToWWToENEN_UL2016, GluGluToWWToENMN_UL2016, GluGluToWWToENTN_UL2016, GluGluToWWToMNEN_UL2016, GluGluToWWToMNMN_UL2016, GluGluToWWToMNTN_UL2016, GluGluToWWToTNEN_UL2016, GluGluToWWToTNMN_UL2016, GluGluToWWToTNTN_UL2016, ST_tW_top_UL2016, 
                  ST_tW_antitop_UL2016, 
                  GluGluHToWWTo2L2Nu_UL2016, GluGluHToZZTo4L_UL2016, GluGluHToTauTau_UL2016, VBFHToWWTo2L2Nu_UL2016, VBFHToTauTau_UL2016, ttHToNonbb_UL2016, VHToNonbb_UL2016], 
    'ZZtoLep': [ZZTo2L2Nu_UL2016, ZZTo4L_UL2016, GluGluToContinToZZTo2e2nu_UL2016, GluGluToContinToZZTo2e2mu_UL2016, GluGluToContinToZZTo2e2tau_UL2016, GluGluToContinToZZTo2mu2nu_UL2016, GluGluToContinToZZTo2mu2tau_UL2016, GluGluToContinToZZTo4e_UL2016, GluGluToContinToZZTo4mu_UL2016, GluGluToContinToZZTo4tau_UL2016], 
    'VBS_SSWW_SM': [VBS_SSWW_LL_SM_UL2016, VBS_SSWW_TL_SM_UL2016, VBS_SSWW_TT_SM_UL2016], 
    'Data': [DataEleF_UL2016, DataEleG_UL2016, DataEleH_UL2016, DataMuF_UL2016, DataMuG_UL2016, DataMuH_UL2016,], 
}

aggregated_samples_UL2016APV = {
    'VG':  [ZG_UL2016APV, WG_UL2016APV],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2016APV],   
    'TVX': [TTGJets_UL2016APV, TTZToQQ_UL2016APV, TTZToLLNuNu_UL2016APV, TTWJetsToQQ_UL2016APV, TTWJetsToLNu_UL2016APV, tZq_ll_4f_UL2016APV], 
    'Triboson': [WWW_4F_UL2016APV, WWZ_4F_UL2016APV, WZZ_UL2016APV, ZZZ_UL2016APV, WWG_UL2016APV], 
    'TTTo2L2Nu': [TTTo2L2Nu_UL2016APV], 
    'WZ': [WZ_UL2016APV], 
    'DYJetsToLL_FxFx': [DYJetsToLL_M50_FxFx_UL2016APV],
    'WrongSign': [WWto2L2Nu_UL2016APV, GluGluToWWToENEN_UL2016APV, GluGluToWWToENMN_UL2016APV, GluGluToWWToENTN_UL2016APV, GluGluToWWToMNEN_UL2016APV, GluGluToWWToMNMN_UL2016APV, GluGluToWWToMNTN_UL2016APV, GluGluToWWToTNEN_UL2016APV, GluGluToWWToTNMN_UL2016APV, GluGluToWWToTNTN_UL2016APV, ST_tW_top_UL2016APV, 
                  ST_tW_antitop_UL2016APV, 
                  GluGluHToWWTo2L2Nu_UL2016APV, GluGluHToZZTo4L_UL2016APV, GluGluHToTauTau_UL2016APV, VBFHToWWTo2L2Nu_UL2016APV, VBFHToTauTau_UL2016APV, ttHToNonbb_UL2016APV, VHToNonbb_UL2016APV], 
    'ZZtoLep': [ZZTo2L2Nu_UL2016APV, ZZTo4L_UL2016APV, GluGluToContinToZZTo2e2nu_UL2016APV, GluGluToContinToZZTo2e2mu_UL2016APV, GluGluToContinToZZTo2e2tau_UL2016APV, GluGluToContinToZZTo2mu2nu_UL2016APV, GluGluToContinToZZTo2mu2tau_UL2016APV, GluGluToContinToZZTo4e_UL2016APV, GluGluToContinToZZTo4mu_UL2016APV, GluGluToContinToZZTo4tau_UL2016APV], 
    'VBS_SSWW_SM': [VBS_SSWW_LL_SM_UL2016APV, VBS_SSWW_TL_SM_UL2016APV, VBS_SSWW_TT_SM_UL2016APV], 
    'Data': [DataEleB2_UL2016APV, DataEleC_UL2016APV, DataEleD_UL2016APV, DataEleE_UL2016APV, DataEleF_UL2016APV, DataMuB2_UL2016APV, DataMuC_UL2016APV, DataMuD_UL2016APV, DataMuE_UL2016APV, DataMuF_UL2016APV], 
}

aggregated_samples_UL2017 = {
    'VG':  [ZG_UL2017, WG_UL2017],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2017],   
    'TVX': [TTGJets_UL2017, TTZToQQ_UL2017, TTZToLLNuNu_UL2017, TTWJetsToQQ_UL2017, TTWJetsToLNu_UL2017, tZq_ll_4f_UL2017], 
    'Triboson': [WWW_4F_UL2017, WWZ_4F_UL2017, WZZ_UL2017, ZZZ_UL2017, WWG_UL2017], 
    'TTTo2L2Nu': [TTTo2L2Nu_UL2017], 
    'WZ': [WZ_UL2017], 
    'DYJetsToLL_FxFx': [DYJetsToLL_M50_FxFx_UL2017],
    'WrongSign': [WWto2L2Nu_UL2017, GluGluToWWToENEN_UL2017, GluGluToWWToENMN_UL2017, GluGluToWWToENTN_UL2017, GluGluToWWToMNEN_UL2017, GluGluToWWToMNMN_UL2017, GluGluToWWToMNTN_UL2017, GluGluToWWToTNEN_UL2017, GluGluToWWToTNMN_UL2017, GluGluToWWToTNTN_UL2017, ST_tW_top_UL2017, 
                  ST_tW_antitop_UL2017, 
                  GluGluHToWWTo2L2Nu_UL2017, GluGluHToZZTo4L_UL2017, GluGluHToTauTau_UL2017, VBFHToWWTo2L2Nu_UL2017, VBFHToTauTau_UL2017, ttHToNonbb_UL2017, VHToNonbb_UL2017], 
    'ZZtoLep': [ZZTo2L2Nu_UL2017, ZZTo4L_UL2017, GluGluToContinToZZTo2e2nu_UL2017, GluGluToContinToZZTo2e2mu_UL2017, GluGluToContinToZZTo2e2tau_UL2017, GluGluToContinToZZTo2mu2nu_UL2017, GluGluToContinToZZTo2mu2tau_UL2017, GluGluToContinToZZTo4e_UL2017, GluGluToContinToZZTo4mu_UL2017, GluGluToContinToZZTo4tau_UL2017], 
    'VBS_SSWW_SM': [VBS_SSWW_LL_SM_UL2017, VBS_SSWW_TL_SM_UL2017, VBS_SSWW_TT_SM_UL2017], 
    'Data': [DataEleB_UL2017, DataEleC_UL2017, DataEleD_UL2017, DataEleE_UL2017, DataEleF_UL2017, DataMuB_UL2017, DataMuC_UL2017, DataMuD_UL2017, DataMuE_UL2017, DataMuF_UL2017], 
}

aggregated_samples_UL2018 = {
    'VG':  [ZG_UL2018, WG_UL2018],
    #'QCD_ssWW': [WpWpJJ_QCD_UL2018],   
    'TVX': [TTGJets_UL2018, TTZToQQ_UL2018, TTZToLLNuNu_UL2018, TTWJetsToQQ_UL2018, TTWJetsToLNu_UL2018, tZq_ll_4f_UL2018], 
    'Triboson': [WWW_4F_UL2018, WWZ_4F_UL2018, WZZ_UL2018, ZZZ_UL2018, WWG_UL2018], 
    'TTTo2L2Nu': [TTTo2L2Nu_UL2018], 
    'WZ': [WZ_UL2018], 
    'DYJetsToLL_FxFx': [DYJetsToLL_M50_FxFx_UL2018],
    'WrongSign': [WWto2L2Nu_UL2018, GluGluToWWToENEN_UL2018, GluGluToWWToENMN_UL2018, GluGluToWWToENTN_UL2018, GluGluToWWToMNEN_UL2018, GluGluToWWToMNMN_UL2018, GluGluToWWToMNTN_UL2018, GluGluToWWToTNEN_UL2018, GluGluToWWToTNMN_UL2018, GluGluToWWToTNTN_UL2018, ST_tW_top_UL2018, 
                  ST_tW_antitop_UL2018, 
                  GluGluHToWWTo2L2Nu_UL2018, GluGluHToZZTo4L_UL2018, GluGluHToTauTau_UL2018, VBFHToWWTo2L2Nu_UL2018, VBFHToTauTau_UL2018, ttHToNonbb_UL2018, VHToNonbb_UL2018], 
    'ZZtoLep': [ZZTo2L2Nu_UL2018, ZZTo4L_UL2018, GluGluToContinToZZTo2e2nu_UL2018, GluGluToContinToZZTo2e2mu_UL2018, GluGluToContinToZZTo2e2tau_UL2018, GluGluToContinToZZTo2mu2nu_UL2018, GluGluToContinToZZTo2mu2tau_UL2018, GluGluToContinToZZTo4e_UL2018, GluGluToContinToZZTo4mu_UL2018, GluGluToContinToZZTo4tau_UL2018], 
    'VBS_SSWW_SM': [VBS_SSWW_LL_SM_UL2018, VBS_SSWW_TL_SM_UL2018, VBS_SSWW_TT_SM_UL2018], 
    'Data': [DataEleA_UL2018, DataEleB_UL2018, DataEleC_UL2018, DataEleD_UL2018, DataMuA_UL2018, DataMuB_UL2018, DataMuC_UL2018, DataMuD_UL2018], 
}

aggregated_samples = aggregated_samples_UL2017

In [11]:
chain = []
chain_data = []
chain_WZ = []
proxy = redirector
max_lines = maxNfilespersample

for v in aggregated_samples.values():
    for s in v:
        with open("macros/files/{}.txt".format(s.name)) as file:
            i = 0
            for line in file:
                if i < max_lines: 
                    line = line.rstrip()
                    splitted_parts = line.split("/")
                    splitted_parts[2] = proxy
                    line = "/".join(splitted_parts)
                    if "Data" in line:
                        chain_data.append(line)
                    elif "/WZ_" in line:
                        chain_WZ.append(line)
                    else:
                        chain.append(line)
                i += 1
#chain

In [12]:
chain_WZ[0]

'root://stormgf2.pi.infn.it//store/user/apiccine/VBSUL/WZ_TuneCP5_13TeV-pythia8/WZ_UL2017/220326_003323/0000/tree_hadd_1.root'

In [13]:
ntasks_e2e = 276
if e2e == True:
    chain = []
    chain_WZ = []
    chain_data = []
    url_e2e = "davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms/store/user/ttedesch/"
    url_e2e = "root://t2-xrdcms.lnl.infn.it:7070//store/user/ttedesch/"
    for i in range(0,ntasks_e2e):
        chain.append(url_e2e + folder_e2e + "/preselectionUL_{}.root".format(i))
        
    for i in range(0,5):
        chain.append(url_e2e + folder_e2e + "/preselectionUL_GluGlu_{}.root".format(i))
        chain_WZ.append(url_e2e + folder_e2e + "/preselectionUL_WZ_{}.root".format(i))
        chain_data.append(url_e2e + folder_e2e_data + "/preselectionUL_dataB_{}.root".format(i))
        chain_data.append(url_e2e + folder_e2e_data + "/preselectionUL_dataC_{}.root".format(i))
        chain_data.append(url_e2e + folder_e2e_data + "/preselectionUL_dataD_{}.root".format(i))
        chain_data.append(url_e2e + folder_e2e_data + "/preselectionUL_dataE_{}.root".format(i))
        chain_data.append(url_e2e + folder_e2e_data + "/preselectionUL_dataF_{}.root".format(i))

# Do analysis

## MC (no WZ)

In [14]:
h = {}

In [15]:
chain[0]

'root://t2-xrdcms.lnl.infn.it:7070//store/user/ttedesch/preselection_UL2017_v24_monitoring_remote_v2_O2_iter6/preselectionUL_0.root'

In [16]:
if distributed == True:
    df = RDataFrame("Events", chain, npartitions=nmaxpartitions, daskclient=client, monitor_label = "main")
else:
    #chain = ["tree_hadd_1.root"] #root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/ZG_UL2017/220227_181212/0000/tree_hadd_1.root
    #chain = ["tree_hadd_1_data.root"] #root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/SingleElectron/DataEleB_UL2017/220228_083837/0000/tree_hadd_1.root
    df = RDataFrame("Events", chain[0])   
    
df_type = df.Define("IsMC", "isMC(Sample)").Define("Year", "\"UL2017\"")
df_tau_SF = produce_tau_SF(df_type)
df_jet_tau_redefines = jet_tau_redefines(df_tau_SF).Define("MET_T1Smear_pt_vec", "RVec<float>{ (float) MET_T1Smear_pt}").Define("MET_T1Smear_phi_vec", "RVec<float>{ (float) MET_T1Smear_phi}")\
                                                   .Define("MET_T1Smear_pt_jerDown_vec", "RVec<float>{ (float) MET_T1Smear_pt_jerDown}").Define("MET_T1Smear_phi_jerDown_vec", "RVec<float>{ (float) MET_T1Smear_phi_jerDown}")\
                                                   .Define("MET_T1Smear_pt_jerUp_vec", "RVec<float>{ (float) MET_T1Smear_pt_jerUp}").Define("MET_T1Smear_phi_jerUp_vec", "RVec<float>{ (float) MET_T1Smear_phi_jerUp}")\
                                                   .Define("MET_T1Smear_pt_jesTotalDown_vec", "RVec<float>{ (float) MET_T1Smear_pt_jesTotalDown}").Define("MET_T1Smear_phi_jesTotalDown_vec", "RVec<float>{ (float) MET_T1Smear_phi_jesTotalDown}")\
                                                   .Define("MET_T1Smear_pt_jesTotalUp_vec", "RVec<float>{ (float) MET_T1Smear_pt_jesTotalUp}").Define("MET_T1Smear_phi_jesTotalUp_vec", "RVec<float>{ (float) MET_T1Smear_phi_jesTotalUp}")
if do_variations == True:
    df_sys = energetic_variations(df_jet_tau_redefines)
else:
    df_sys = df_jet_tau_redefines


#trigger and VBS jets selections  and e/mu + tau final state
df_trigger = filter_trigger(df_sys)
df_jets = select_jets(df_trigger)
df_lepton = select_lepton(df_jets).Define("PassDataLeptonCheck", "DataLeptonCheck(Sample, GoodLeptonFamily, IsMC)").Filter("PassDataLeptonCheck == true", "DataLeptonCheck")\
                                  .Redefine("Tau_pt", "Tau_pt*FESSF*TESSF")\
                                  .Redefine("Tau_mass", "Tau_mass*FESSF*TESSF")\

df_tau = select_tau(df_lepton).Define("lepton_genPartFlav", "GetLepton(Electron_genPartFlav, Electron_idx, Muon_genPartFlav, Muon_idx, GoodLeptonFamily)")\
                              .Define("tau_genPartFlav", "GetTau(Tau_genPartFlav, Tau_idx)")\
                              .Define("lepton_isPrompt", "lepton_genPartFlav")\
                              .Define("tau_isPrompt", "tau_genPartFlav")

#df_corrections = corrections(df_tau)
df_definitions = general_definitions(df_tau)
df_SF_def= scale_factors(df_definitions).Define("pdf_total_weights", "PdfWeight_variations(LHEPdfWeight, Generator_weight, Sample)")\
                                        .Define("pdf_totalSF", "pdf_total_weights[0]")\
                                        .Define("pdf_totalUp", "pdf_total_weights[1]")\
                                        .Define("pdf_totalDown", "pdf_total_weights[2]")\
                                        .Define("QCDScale_weights", "QCDScale_variations(LHEScaleWeight)")\
                                        .Define("QCDScaleSF", "QCDScale_weights[0]")\
                                        .Define("QCDScaleUp", "QCDScale_weights[1]")\
                                        .Define("QCDScaleDown", "QCDScale_weights[2]")\
                                        .Define("ISRSF", "1.f")\
                                        .Define("FSRSF", "1.f")\
                                        .Define("PSWeight_weights", "PSWeight_variations(PSWeight)")\
                                        .Define("ISRUp", "PSWeight_weights[1]")\
                                        .Define("ISRDown", "PSWeight_weights[0]")\
                                        .Define("FSRUp", "PSWeight_weights[3]")\
                                        .Define("FSRDown", "PSWeight_weights[2]")\
        
if do_variations == True:
    df_SF_var = SF_variations(df_SF_def)
else:
    df_SF_var = df_SF_def

df_SF = df_SF_var.Define("w_SF", "PrefireWeight*puWeight*lepSF*tau_vsjet_SF*tau_vsele_SF*tau_vsmu_SF*btagSF*pdf_totalSF*QCDScaleSF*ISRSF*FSRSF")\
                 .Define("Nevents", "getNevents(Sample, IsMC)")\
                 .Define("HLT_effLumi", "getLumi(Year, IsMC)")\
                 .Define("xsec", "getXSec(Sample, IsMC)")\
                 .Define("w_nominal", "HLT_effLumi*xsec* 1000./Nevents")\
                 .Define("wzero", "w_SF*w_nominal")

#lepton_isPrompt,lepton_pdgid,mT_leptau_MET,mT_tau_MET,ptRel_jj,puDown,puSF,puUp,tauDown,tauUp,tau_DecayMode,tau_GenMatch,tau_isPrompt,tauleadTk_ptOverTau

### book snapshot ####
opts = ROOT.RDF.RSnapshotOptions()
opts.fLazy = True

if do_variations == False:
    df_SF_lazySnap = df_SF.Snapshot(outTreeName, outFilePath, MC_columns, opts)

df_SS = df_SF.Filter("pass_charge_selection == true", "SameSign")
df_OS = df_SF.Filter("pass_charge_selection == false", "OppositeSign")

df_SS_METCut_true = df_SS.Filter("pass_MET_cut == true", "MET cut true")
df_SS_METCut_false = df_SS.Filter("pass_MET_cut == false", "MET cut false")

df_OS_METCut_true = df_OS.Filter("pass_MET_cut == true", "MET cut true")

df_SS_METCut_true_bjets_true = df_SS_METCut_true.Filter("pass_b_veto == false", "Bveto")#.Filter(".Filter("nBJet > 0", "at least one b jets - medium")
df_SS_METCut_true_bjets_false = df_SS_METCut_true.Filter("pass_b_veto == true", "Bveto")#.Filter("nBJet == 0", "no b jets - medium")

df_OS_METCut_true_bjets_true = df_OS_METCut_true.Filter("pass_b_veto == false", "Bveto")#.Filter("nBJet > 0", "at least one b jets - medium")

to_plot_ = []
for item in aggregated_samples.items():
    if not item[0] in ["Fake", "Data", "WZ"]:
        to_plot_.append(item[0])
##### CR fakes
df_FAKES = df_SS_METCut_false
book_histos(df_FAKES, 'fakes_CR', h, to_plot = to_plot_)
##### SR 
df_SR = df_SS_METCut_true_bjets_false.Filter("pass_mjj_cut == true", "mjj cut")
book_histos(df_SR, 'SR', h, to_plot = to_plot_)
#### CR ttbar
df_TTBAR = df_OS_METCut_true_bjets_true
book_histos(df_TTBAR, 'ttbar_CR', h, to_plot = to_plot_)
#### CR opposite sign 
df_OPPOSITESIGN = df_OS.Filter("pass_b_veto_loose == true", "Bveto")
book_histos(df_OPPOSITESIGN, 'OS_CR_bvetoL', h, to_plot = to_plot_)


/usr/local/lib/python3.8/dist-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [17]:
if do_variations == True:
    print("sono qui")
    h_varied = {}
    for region, samples_dict in h.items():
        h_varied[region] = {}
        for sample, histos_dict in samples_dict.items(): 
            h_varied[region][sample] = {}
            for feature, histo in histos_dict.items():
                h_varied[region][sample][feature] = {}
                for final_state in ['etau', 'mutau']:
                    #print(type(histo))
                    #print(type(histo[final_state]))
                    if distributed == True:
                        h_varied[region][sample][feature][final_state] = ROOT.RDF.Experimental.Distributed.VariationsFor(histo[final_state])
                    else:
                        h_varied[region][sample][feature][final_state] = ROOT.RDF.Experimental.VariationsFor(histo[final_state])

sono qui


In [18]:
'''
%%time
if do_variations == False:
    df_SF_lazySnap.GetValue()
'''

'\n%%time\nif do_variations == False:\n    df_SF_lazySnap.GetValue()\n'

In [19]:
'''
%%time
if do_variations == True:
    h_varied["OS_CR_bvetoL"]["VG"]["m_jj"]["etau"]["jer:up"].Draw()
'''

'\n%%time\nif do_variations == True:\n    h_varied["OS_CR_bvetoL"]["VG"]["m_jj"]["etau"]["jer:up"].Draw()\n'

In [20]:
'''
%%time
client.run(transfer_to_tier)
'''

'\n%%time\nclient.run(transfer_to_tier)\n'

In [21]:
#!davix-ls -l davs://t2-xrdcms.lnl.infn.it:2880/pnfs/lnl.infn.it/data/cms/store/user/ttedesch/postselection_bench_v20_1 -E /opt/workspace/persistent-storage/proxy --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/

In [22]:
#allCutsReport = df.Report()
#allCutsReport.Print()

## WZ

In [23]:
h_WZ = {}

In [24]:
if distributed == True:
    #df_WZ = RDataFrame("Events", chain_WZ[0], npartitions=1, daskclient=client)
    df_WZ = RDataFrame("Events", chain_WZ, npartitions=5, daskclient=client, monitor_label = "WZ")
else:
    #chain = ["tree_hadd_1.root"] #root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/ZG_UL2017/220227_181212/0000/tree_hadd_1.root
    #chain = ["tree_hadd_1_data.root"] #root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/SingleElectron/DataEleB_UL2017/220228_083837/0000/tree_hadd_1.root
    df_WZ = RDataFrame("Events", chain_WZ[0]).Range(1000)    
    
df_type_WZ = df_WZ.Define("IsMC", "isMC(Sample)").Define("Year", "\"UL2017\"")
df_tau_SF_WZ = produce_tau_SF(df_type_WZ)
df_jet_tau_redefines_WZ = jet_tau_redefines(df_tau_SF_WZ).Define("MET_T1Smear_pt_vec", "RVec<float>{ (float) MET_T1Smear_pt}").Define("MET_T1Smear_phi_vec", "RVec<float>{ (float) MET_T1Smear_phi}")\
                                                         .Define("MET_T1Smear_pt_jerDown_vec", "RVec<float>{ (float) MET_T1Smear_pt_jerDown}").Define("MET_T1Smear_phi_jerDown_vec", "RVec<float>{ (float) MET_T1Smear_phi_jerDown}")\
                                                         .Define("MET_T1Smear_pt_jerUp_vec", "RVec<float>{ (float) MET_T1Smear_pt_jerUp}").Define("MET_T1Smear_phi_jerUp_vec", "RVec<float>{ (float) MET_T1Smear_phi_jerUp}")\
                                                         .Define("MET_T1Smear_pt_jesTotalDown_vec", "RVec<float>{ (float) MET_T1Smear_pt_jesTotalDown}").Define("MET_T1Smear_phi_jesTotalDown_vec", "RVec<float>{ (float) MET_T1Smear_phi_jesTotalDown}")\
                                                         .Define("MET_T1Smear_pt_jesTotalUp_vec", "RVec<float>{ (float) MET_T1Smear_pt_jesTotalUp}").Define("MET_T1Smear_phi_jesTotalUp_vec", "RVec<float>{ (float) MET_T1Smear_phi_jesTotalUp}")

if do_variations == True:
    df_sys_WZ = energetic_variations(df_jet_tau_redefines_WZ)
else:
    df_sys_WZ = df_jet_tau_redefines_WZ

#trigger and VBS jets selections  and e/mu + tau final state
df_trigger_WZ = filter_trigger(df_sys_WZ)
df_jets_WZ = select_jets(df_trigger_WZ)
df_lepton_WZ = select_lepton(df_jets_WZ).Define("PassDataLeptonCheck", "DataLeptonCheck(Sample, GoodLeptonFamily, IsMC)").Filter("PassDataLeptonCheck == true", "DataLeptonCheck")\
                                  .Redefine("Tau_pt", "Tau_pt*FESSF*TESSF")\
                                  .Redefine("Tau_mass", "Tau_mass*FESSF*TESSF")\

df_tau_WZ = select_tau(df_lepton_WZ).Define("lepton_genPartFlav", "GetLepton(Electron_genPartFlav, Electron_idx, Muon_genPartFlav, Muon_idx, GoodLeptonFamily)")\
                              .Define("tau_genPartFlav", "GetTau(Tau_genPartFlav, Tau_idx)")\
                              .Define("lepton_isPrompt", "lepton_genPartFlav")\
                              .Define("tau_isPrompt", "tau_genPartFlav")

#df_corrections = corrections(df_tau)
df_definitions_WZ = general_definitions(df_tau_WZ)
df_SF_def_WZ= scale_factors(df_definitions_WZ).Define("pdf_totalSF", "1.f")\
                                        .Define("pdf_totalUp", "1.f")\
                                        .Define("pdf_totalDown", "1.f")\
                                        .Define("QCDScaleSF", "1.f")\
                                        .Define("QCDScaleUp", "1.f")\
                                        .Define("QCDScaleDown", "1.f")\
                                        .Define("ISRSF", "1.f")\
                                        .Define("FSRSF", "1.f")\
                                        .Define("PSWeight_weights", "PSWeight_variations(PSWeight)")\
                                        .Define("ISRUp", "PSWeight_weights[1]")\
                                        .Define("ISRDown", "PSWeight_weights[0]")\
                                        .Define("FSRUp", "PSWeight_weights[3]")\
                                        .Define("FSRDown", "PSWeight_weights[2]")\
        
if do_variations == True:
    df_SF_var_WZ = SF_variations(df_SF_def_WZ)
else:
    df_SF_var_WZ = df_SF_def_WZ

df_SF_WZ = df_SF_var_WZ.Define("w_SF", "PrefireWeight*puWeight*lepSF*tau_vsjet_SF*tau_vsele_SF*tau_vsmu_SF*btagSF*pdf_totalSF*QCDScaleSF*ISRSF*FSRSF")\
                 .Define("Nevents", "getNevents(Sample, IsMC)")\
                 .Define("HLT_effLumi", "getLumi(Year, IsMC)")\
                 .Define("xsec", "getXSec(Sample, IsMC)")\
                 .Define("w_nominal", "HLT_effLumi*xsec* 1000./Nevents")\
                 .Define("wzero", "w_SF*w_nominal")

#lepton_isPrompt,lepton_pdgid,mT_leptau_MET,mT_tau_MET,ptRel_jj,puDown,puSF,puUp,tauDown,tauUp,tau_DecayMode,tau_GenMatch,tau_isPrompt,tauleadTk_ptOverTau

### book snapshot ####
opts = ROOT.RDF.RSnapshotOptions()
opts.fLazy = True

if do_variations == False:
    df_SF_lazySnap_WZ = df_SF_WZ.Snapshot(outTreeName, outFilePath_WZ, MC_columns, opts)

df_SS_WZ = df_SF_WZ.Filter("pass_charge_selection == true", "SameSign")
df_OS_WZ = df_SF_WZ.Filter("pass_charge_selection == false", "OppositeSign")

df_SS_METCut_true_WZ = df_SS_WZ.Filter("pass_MET_cut == true", "MET cut true")
df_SS_METCut_false_WZ = df_SS_WZ.Filter("pass_MET_cut == false", "MET cut false")

df_OS_METCut_true_WZ = df_OS_WZ.Filter("pass_MET_cut == true", "MET cut true")

df_SS_METCut_true_bjets_true_WZ = df_SS_METCut_true_WZ.Filter("pass_b_veto == false", "Bveto")#.Filter(".Filter("nBJet > 0", "at least one b jets - medium")
df_SS_METCut_true_bjets_false_WZ = df_SS_METCut_true_WZ.Filter("pass_b_veto == true", "Bveto")#.Filter("nBJet == 0", "no b jets - medium")

df_OS_METCut_true_bjets_true_WZ = df_OS_METCut_true_WZ.Filter("pass_b_veto == false", "Bveto")#.Filter("nBJet > 0", "at least one b jets - medium")

##### CR fakes
df_FAKES_WZ = df_SS_METCut_false_WZ
book_histos(df_FAKES_WZ, 'fakes_CR', h_WZ, to_plot = ["WZ"])
##### SR 
df_SR_WZ = df_SS_METCut_true_bjets_false_WZ.Filter("pass_mjj_cut == true", "mjj cut")
book_histos(df_SR_WZ, 'SR', h_WZ, to_plot = ["WZ"])
#### CR ttbar
df_TTBAR_WZ = df_OS_METCut_true_bjets_true_WZ
book_histos(df_TTBAR_WZ, 'ttbar_CR', h_WZ, to_plot = ["WZ"])
#### CR opposite sign 
df_OPPOSITESIGN_WZ = df_OS_WZ.Filter("pass_b_veto_loose == true", "Bveto")
book_histos(df_OPPOSITESIGN_WZ, 'OS_CR_bvetoL', h_WZ, to_plot = ["WZ"])

In [25]:
if do_variations == True:
    print("sono qui")
    h_varied_WZ = {}
    for region, samples_dict in h_WZ.items():
        h_varied_WZ[region] = {}
        for sample, histos_dict in samples_dict.items(): 
            h_varied_WZ[region][sample] = {}
            for feature, histo in histos_dict.items():
                h_varied_WZ[region][sample][feature] = {}
                for final_state in ['etau', 'mutau']:
                    #print(type(histo))
                    #print(type(histo[final_state]))
                    if distributed == True:
                        h_varied_WZ[region][sample][feature][final_state] = ROOT.RDF.Experimental.Distributed.VariationsFor(histo[final_state])
                    else:
                        h_varied_WZ[region][sample][feature][final_state] = ROOT.RDF.Experimental.VariationsFor(histo[final_state])

sono qui


In [26]:
'''
%%time
c = ROOT.TCanvas()
h_varied["OS_CR_bvetoL"]["VG"]["mjj"]["etau"]["jer:up"].Draw()
c.Draw()
'''

'\n%%time\nc = ROOT.TCanvas()\nh_varied["OS_CR_bvetoL"]["VG"]["mjj"]["etau"]["jer:up"].Draw()\nc.Draw()\n'

In [27]:
'''
%%time
c = ROOT.TCanvas()
h_varied_WZ["OS_CR_bvetoL"]["WZ"]["mjj"]["etau"]["jer:up"].Draw()
c.Draw()
'''

'\n%%time\nc = ROOT.TCanvas()\nh_varied_WZ["OS_CR_bvetoL"]["WZ"]["mjj"]["etau"]["jer:up"].Draw()\nc.Draw()\n'

In [28]:

%%time
if distributed == True:
    RunGraphs = ROOT.RDF.Experimental.Distributed.RunGraphs   
else:
    RunGraphs = ROOT.RDF.RunGraphs

if do_variations == False:
    proxies = [df_SF_lazySnap, df_SF_lazySnap_WZ]
    RunGraphs(proxies)
    dfs = [df_.GetValue() for df_ in proxies]
"""
if do_variations == True:
    proxies = [h_varied["OS_CR_bvetoL"]["VG"]["m_jj"]["etau"]["jer:up"], h_varied_WZ["OS_CR_bvetoL"]["WZ"]["m_jj"]["etau"]["jer:up"]]
    RunGraphs(proxies)
    dfs = [df_.Draw() for df_ in proxies]  
"""

CPU times: user 933 µs, sys: 40 µs, total: 973 µs
Wall time: 999 µs


'\nif do_variations == True:\n    proxies = [h_varied["OS_CR_bvetoL"]["VG"]["m_jj"]["etau"]["jer:up"], h_varied_WZ["OS_CR_bvetoL"]["WZ"]["m_jj"]["etau"]["jer:up"]]\n    RunGraphs(proxies)\n    dfs = [df_.Draw() for df_ in proxies]  \n'

In [29]:
ERROR

NameError: name 'ERROR' is not defined

In [30]:
%%time
import math

if results_dir not in os.listdir():
    os.mkdir("./" + results_dir)

if do_variations == True:
    variations = ["nominal", "jesTotal", "jer", "TES", "FES", "PF", "pu", "btag", "lep", "tau_vsjet", "tau_vsele", "tau_vsmu", "pdf_total", "QCDScale", "ISR", "FSR"]
else: 
    variations = ["nominal"]
    #variations = ["PF", "pu", "btag", "lep", "tau_vsjet", "tau_vsele", "tau_vsmu"]

if "plots" not in os.listdir("./" + results_dir):
    os.mkdir("./" + results_dir +"/" + "plots")
folder = "./"+ results_dir + "/plots"
if "electron" not in os.listdir(folder):
    os.mkdir(folder + "/" + "electron")
if "muon" not in os.listdir(folder):
    os.mkdir(folder + "/" + "muon")

"""OLD NOT UPDATED
def write_histo(h_, to_plot, isData = False):
    for lepton, final_state in zip(['electron', 'muon'], ['etau', 'mutau']):
        for region, samples_dict in h_.items():
            print(region, final_state)
            for sample, histos_dict in samples_dict.items():
                if not sample in to_plot:
                    continue
                file = ROOT.TFile(folder + "/" + lepton + "/" + "{}.root".format(sample), 'UPDATE')
                for n, var in enumerate(variations):
                    for feature, histo in histos_dict.items():
                        #print(region, final_state, sample, feature, var)
                        #histo[final_state].Write()
                        if n == 0:
                            if isData == False:
                                if do_variations == False:
                                    h1 = histo[final_state].GetValue() 
                                else:
                                    h1 = histo[final_state]['nominal']
                            else:
                                h1 = histo[final_state].GetValue()
                            nbins = h1.GetNbinsX()
                            for bidx in range(nbins):          
                                bidx_l = bidx + 1
                                #print("qui")
                                if h1.GetBinContent(bidx_l) < 0:
                                    h1.SetBinContent(bidx_l, 0.)
                                #print("qsaasdsdadai")
                            #print("slkncalk")
                            h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                            #print("qui")
                            h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                            h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                            h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                            if sample == "Fake":
                                for bidxx in range(nbins):          
                                    bidxx_l = bidxx + 1
                                    h1.SetBinError(bidxx_l, 0.3*h1.GetBinContent(bidxx_l))
                            h1.Write()
                        else:
                            if isData == False:
                                for var_type in ['up','down']:
                                    h1 = histo[final_state][var + ":" + var_type]
                                    h1.SetName(h1.GetName() + "_" + var + var_type.capitalize())
                                    nbins = h1.GetNbinsX()
                                    for bidx in range(nbins):          
                                        bidx_l = bidx + 1
                                        if h1.GetBinContent(bidx_l) < 0:
                                            h1.SetBinContent(bidx_l, 0.)
                                    h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                                    h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                                    h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                                    h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                                    if sample == "Fake":
                                        for bidx in range(nbins):          
                                            bidx_l = bidx + 1
                                            h1.SetBinError(bidx_l, 0.3*h1.GetBinContent(bidx_l))
                                    h1.Write()
                file.Close()
    print()
"""
def write_histo(h_, to_plot, isData = False):
    for lepton, final_state in zip(['electron', 'muon'], ['etau', 'mutau']):
        for region, samples_dict in h_.items():
            print(region, final_state)
            for sample, histos_dict in samples_dict.items():
                if not sample in to_plot:
                    continue
                file = ROOT.TFile(folder + "/" + lepton + "/" + "{}.root".format(sample), 'UPDATE')
                for n, var in enumerate(variations):
                    for feature, histo in histos_dict.items():
                        #print(region, final_state, sample, feature, var)
                        #histo[final_state].Write()
                        if n == 0:
                            if isData == False:
                                if do_variations == False:
                                    h1 = histo[final_state].GetValue() 
                                else:
                                    h1 = histo[final_state]['nominal']
                            else:
                                h1 = histo[final_state].GetValue()
                            nbins = h1.GetNbinsX()
                            #print("qsaasdsdadai")
                            #print("slkncalk")
                            h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                            #print("qui")
                            h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                            h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                            h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                            for i in range(0, nbins + 1):          
                                #bidx_l = bidx + 1
                                #print("qui")
                                if h1.GetBinContent(i) < 0:
                                    h1.SetBinContent(i, 0.)
                            if sample == "Fake":
                                for bidxx in range(nbins):          
                                    bidxx_l = bidxx + 1
                                    h1.SetBinError(bidxx_l, 0.3*h1.GetBinContent(bidxx_l))
                            h1.Write()
                        else:
                            if isData == False:
                                for var_type in ['up','down']:
                                    h1 = histo[final_state][var + ":" + var_type]
                                    h1.SetName(h1.GetName() + "_" + var + var_type.capitalize())
                                    nbins = h1.GetNbinsX()
                                    h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                                    h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                                    h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                                    h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                                    for i in range(0, nbins + 1):          
                                        #bidx_l = bidx + 1
                                        if h1.GetBinContent(i) < 0:
                                            h1.SetBinContent(i, 0.)
                                    if sample == "Fake":
                                        for bidx in range(nbins):          
                                            bidx_l = bidx + 1
                                            h1.SetBinError(bidx_l, 0.3*h1.GetBinContent(bidx_l))
                                    h1.Write()
                file.Close()
    print()



if do_variations == True:
    write_histo(h_varied, [s_ for s_ in aggregated_samples.keys() if not s_ in ["WZ", "Fake", "Data"]])
else:
    write_histo(h, [s_ for s_ in aggregated_samples.keys() if not s_ in ["WZ", "Fake", "Data"]])


fakes_CR etau
SR etau
ttbar_CR etau
OS_CR_bvetoL etau
fakes_CR mutau
SR mutau
ttbar_CR mutau
OS_CR_bvetoL mutau

CPU times: user 13.2 s, sys: 1.46 s, total: 14.6 s
Wall time: 12min 58s


In [31]:
%%time
if do_variations == True:
    write_histo(h_varied_WZ, ["WZ"])
else:
    write_histo(h_WZ, ["WZ"])

fakes_CR etau
SR etau
ttbar_CR etau
OS_CR_bvetoL etau
fakes_CR mutau
SR mutau
ttbar_CR mutau
OS_CR_bvetoL mutau

CPU times: user 327 ms, sys: 57.7 ms, total: 384 ms
Wall time: 10.5 s


In [ ]:
ERROR

In [ ]:
%%time
client.run(transfer_to_tier)

In [ ]:
ERROR

## Data

In [ ]:
h_data = {}

In [ ]:

if distributed == True:
    df_data = RDataFrame("Events", chain_data, npartitions=nmaxpartitions, daskclient=client)
else:
    #chain = ["tree_hadd_1.root"] #root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/ZG_UL2017/220227_181212/0000/tree_hadd_1.root
    #chain = ["tree_hadd_1_data.root"] #root://xrootd-cms.infn.it//store/user/apiccine/ULVBS/SingleElectron/DataEleB_UL2017/220228_083837/0000/tree_hadd_1.root
    df_data = RDataFrame("Events", chain_data[0]).Range(1000)    
    
df_type_data = df_data.Define("IsMC", "isMC(Sample)").Define("Year", "\"UL2017\"").Filter("Flag_eeBadScFilter == true", "Flag_eeBadScFilter")#.Define("MET_T1_pt", "MET_T1_pt_nom").Define("MET_T1_phi", "MET_T1_phi_nom")

'''
df_tau_SF_data = produce_tau_SF(df_type_data)
df_jet_tau_redefines_data = jet_tau_redefines(df_tau_SF_data)
'''
df_jet_tau_redefines_data = jet_tau_redefines(df_type_data)

#trigger and VBS jets selections  and e/mu + tau final state
df_trigger_data = filter_trigger(df_jet_tau_redefines_data)
df_jets_data = select_jets(df_trigger_data)
df_lepton_data = select_lepton(df_jets_data).Define("PassDataLeptonCheck", "DataLeptonCheck(Sample, GoodLeptonFamily, IsMC)").Filter("PassDataLeptonCheck == true", "DataLeptonCheck")\
                                            .Define("lepton_SFFake_vsjet2", "SFFakeRatio_lep_calc_vsjet2(lepton_pt, lepton_eta, lepton_pdgid)")\
                                            .Define("lepton_SFFake_vsjet4", "SFFakeRatio_lep_calc_vsjet4(lepton_pt, lepton_eta, lepton_pdgid)")

df_tau_data = select_tau(df_lepton_data).Define("tau_SFFake_vsjet2", "SFFakeRatio_tau_calc_vsjet2(tau_pt, tau_eta)")\
                                        .Define("tau_SFFake_vsjet4", "SFFakeRatio_tau_calc_vsjet4(tau_pt, tau_eta)")
#df_corrections = corrections(df_tau)
df_definitions_data = general_definitions_data(df_tau_data).Define("event_SFFake_vsjet2", "lepton_SFFake_vsjet2*tau_SFFake_vsjet2")\
                                                           .Define("event_SFFake_vsjet4", "lepton_SFFake_vsjet4*tau_SFFake_vsjet4")

'''
df_SF_def_data= scale_factors(df_definitions_data)


df_SF_data = df_SF_def_data.Define("w_SF", "PrefireWeight*puWeight*LepSF*Tau_vsjet_SF*Tau_vsele_SF*Tau_vsmu_SF*btagSF")\
                           .Define("Nevents", "getNevents(Sample, IsMC)")\
                           .Define("lumi", "getLumi(Year, IsMC)")\
                           .Define("xsec", "getXSec(Sample, IsMC)")\
                           .Define("w_nominal", "lumi*xsec* 1000./Nevents")\
                           .Define("wzero", "w_SF*w_nominal")
'''

df_SF_data = df_definitions_data.Define("wzero", "1.")

df_SS_data = df_SF_data.Filter("pass_charge_selection == true", "SameSign")
df_OS_data = df_SF_data.Filter("pass_charge_selection == false", "OppositeSign")

df_SS_METCut_true_data = df_SS_data.Filter("pass_MET_cut == true", "MET cut true")
df_SS_METCut_false_data = df_SS_data.Filter("pass_MET_cut == false", "MET cut false")

df_OS_METCut_true_data = df_OS_data.Filter("pass_MET_cut == true", "MET cut true")

df_SS_METCut_true_bjets_true_data = df_SS_METCut_true_data.Filter("pass_b_veto == false", "Bveto")#.Filter("nBJet > 0", "at least one b jets - medium")
df_SS_METCut_true_bjets_false_data = df_SS_METCut_true_data.Filter("pass_b_veto == true", "Bveto")#.Filter("nBJet == 0", "no b jets - medium")

df_OS_METCut_true_bjets_true_data = df_OS_METCut_true_data.Filter("pass_b_veto == false", "Bveto")#.Filter("nBJet > 0", "at least one b jets - medium")

##### CR fakes
df_FAKES_data = df_SS_METCut_false_data
book_histos(df_FAKES_data, 'fakes_CR', h_data, to_plot = ["Data"])
##### SR 
df_SR_data = df_SS_METCut_true_bjets_false_data.Filter("pass_mjj_cut == true", "mjj cut")
book_histos(df_SR_data, 'SR', h_data, to_plot =  ["Data"])
#### CR ttbar
df_TTBAR_data = df_OS_METCut_true_bjets_true_data
book_histos(df_TTBAR_data, 'ttbar_CR', h_data, to_plot = ["Data"])
#### CR opposite sign 
df_OPPOSITESIGN_data = df_OS_data.Filter("pass_b_veto_loose == true", "Bveto")
book_histos(df_OPPOSITESIGN_data, 'OS_CR_bvetoL', h_data, to_plot = ["Data"])

In [ ]:
c = ROOT.TCanvas()
h_data["OS_CR_bvetoL"]["Data"]["lepton_pt"]["etau"].Draw()
c.Draw()

In [ ]:
ERROR

# Trigger the three event loops concurrently

In [ ]:
#if distributed == True:
#    RunGraphs = ROOT.RDF.Experimental.Distributed.RunGraphs   
#else:
#    RunGraphs = ROOT.RDF.RunGraphs

#if do_variations == True:
#    proxies = [h_varied["CR_fakes"]["Vgamma"]["mjj"]["etau"]["nominal"], h_data["CR_fakes"]["Vgamma"]["mjj"]["etau"]]
#else:
#    proxies = [h["CR_fakes"]["Vgamma"]["mjj"]["etau"], h_data["CR_fakes"]["Vgamma"]["mjj"]["etau"]]
#RunGraphs(proxies)
#hs_ = [h_.GetValue() for h_ in proxies]

In [ ]:
#from dask.distributed import performance_report
#with performance_report(filename="dask-report_WZ_test.html"):
#    c = ROOT.TCanvas()
#    h_WZ["OS_CR_bvetoL"]["WrongSign"]["mjj"]["etau"].Draw()
#    c.Draw()

In [ ]:
#ERROR GENERATOR

# Save histos

In [ ]:
c = ROOT.TCanvas()
h["SR"]["VG"]["m_jj"]["etau"].Draw()
c.Draw()

In [ ]:
if results_dir not in os.listdir():
    os.mkdir("./" + results_dir)

In [ ]:
if do_variations == True:
    variations = ["nominal", "jesTotal", "jer", "TES", "FES", "PF", "pu", "btag", "lep", "tau_vsjet", "tau_vsele", "tau_vsmu", "pdf_total", "QCDScale", "ISR", "FSR"]
else: 
    variations = ["nominal"]
    #variations = ["PF", "pu", "btag", "lep", "tau_vsjet", "tau_vsele", "tau_vsmu"]

In [ ]:
%%time
import math

if "plots" not in os.listdir("./" + results_dir):
    os.mkdir("./" + results_dir +"/" + "plots")
folder = "./"+ results_dir + "/plots"
if "electron" not in os.listdir(folder):
    os.mkdir(folder + "/" + "electron")
if "muon" not in os.listdir(folder):
    os.mkdir(folder + "/" + "muon")
""" OLD NOT FIXED
def write_histo(h_, to_plot, isData = False):
    for lepton, final_state in zip(['electron', 'muon'], ['etau', 'mutau']):
        for region, samples_dict in h_.items():
            print(region, final_state)
            for sample, histos_dict in samples_dict.items():
                if not sample in to_plot:
                    continue
                file = ROOT.TFile(folder + "/" + lepton + "/" + "{}.root".format(sample), 'UPDATE')
                for n, var in enumerate(variations):
                    for feature, histo in histos_dict.items():
                        #print(region, final_state, sample, feature, var)
                        #histo[final_state].Write()
                        if n == 0:
                            if isData == False:
                                if do_variations == False:
                                    h1 = histo[final_state].GetValue() 
                                else:
                                    h1 = histo[final_state]['nominal']
                            else:
                                h1 = histo[final_state].GetValue()
                            nbins = h1.GetNbinsX()
                            for bidx in range(nbins):          
                                bidx_l = bidx + 1
                                #print("qui")
                                if h1.GetBinContent(bidx_l) < 0:
                                    h1.SetBinContent(bidx_l, 0.)
                                #print("qsaasdsdadai")
                            #print("slkncalk")
                            h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                            #print("qui")
                            h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                            h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                            h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                            if sample == "Fake":
                                for bidxx in range(nbins):          
                                    bidxx_l = bidxx + 1
                                    h1.SetBinError(bidxx_l, 0.3*h1.GetBinContent(bidxx_l))
                            h1.Write()
                        else:
                            if isData == False:
                                for var_type in ['up','down']:
                                    h1 = histo[final_state][var + ":" + var_type]
                                    h1.SetName(h1.GetName() + "_" + var + var_type.capitalize())
                                    nbins = h1.GetNbinsX()
                                    for bidx in range(nbins):          
                                        bidx_l = bidx + 1
                                        if h1.GetBinContent(bidx_l) < 0:
                                            h1.SetBinContent(bidx_l, 0.)
                                    h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                                    h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                                    h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                                    h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                                    if sample == "Fake":
                                        for bidx in range(nbins):          
                                            bidx_l = bidx + 1
                                            h1.SetBinError(bidx_l, 0.3*h1.GetBinContent(bidx_l))
                                    h1.Write()
                file.Close()
    print()
"""
def write_histo(h_, to_plot, isData = False):
    for lepton, final_state in zip(['electron', 'muon'], ['etau', 'mutau']):
        for region, samples_dict in h_.items():
            print(region, final_state)
            for sample, histos_dict in samples_dict.items():
                if not sample in to_plot:
                    continue
                file = ROOT.TFile(folder + "/" + lepton + "/" + "{}.root".format(sample), 'UPDATE')
                for n, var in enumerate(variations):
                    for feature, histo in histos_dict.items():
                        #print(region, final_state, sample, feature, var)
                        #histo[final_state].Write()
                        if n == 0:
                            if isData == False:
                                if do_variations == False:
                                    h1 = histo[final_state].GetValue() 
                                else:
                                    h1 = histo[final_state]['nominal']
                            else:
                                h1 = histo[final_state].GetValue()
                            nbins = h1.GetNbinsX()
                            #print("slkncalk")
                            h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                            #print("qui")
                            h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                            h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                            h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                            for i in range(0, nbins+1):          
                                #bidx_l = bidx + 1
                                #print("qui")
                                if h1.GetBinContent(i) < 0:
                                    h1.SetBinContent(i, 0.)
                                #print("qsaasdsdadai")
                            if sample == "Fake":
                                for bidxx in range(nbins):          
                                    bidxx_l = bidxx + 1
                                    h1.SetBinError(bidxx_l, 0.3*h1.GetBinContent(bidxx_l))
                            h1.Write()
                        else:
                            if isData == False:
                                for var_type in ['up','down']:
                                    h1 = histo[final_state][var + ":" + var_type]
                                    h1.SetName(h1.GetName() + "_" + var + var_type.capitalize())
                                    nbins = h1.GetNbinsX()
                                    h1.SetBinContent(1, h1.GetBinContent(0) + h1.GetBinContent(1))
                                    h1.SetBinError(1, math.sqrt(pow(h1.GetBinError(0),2) + pow(h1.GetBinError(1),2)))
                                    h1.SetBinContent(nbins, h1.GetBinContent(nbins) + h1.GetBinContent(nbins+1))
                                    h1.SetBinError(nbins, math.sqrt(pow(h1.GetBinError(nbins),2) + pow(h1.GetBinError(nbins+1),2)))
                                    for i in range(0, nbins+1):          
                                        #bidx_l = bidx + 1
                                        if h1.GetBinContent(i) < 0:
                                            h1.SetBinContent(i, 0.)
                                    if sample == "Fake":
                                        for bidx in range(nbins):          
                                            bidx_l = bidx + 1
                                            h1.SetBinError(bidx_l, 0.3*h1.GetBinContent(bidx_l))
                                    h1.Write()
                file.Close()
    print()
                

#

In [ ]:
%%time
if do_variations == True:
    write_histo(h_varied, [s_ for s_ in aggregated_samples.keys() if not s_ in ["WZ", "Fake", "Data"]])
else:
    write_histo(h, [s_ for s_ in aggregated_samples.keys() if not s_ in ["WZ", "Fake", "Data"]])

In [ ]:
%%time
if do_variations == True:
    write_histo(h_varied_WZ, ["WZ"])
else:
    write_histo(h_WZ, ["WZ"])

In [ ]:
ERROR

In [ ]:
%%time
write_histo(h_data, ["Fake", "Data"], isData = True)

# PRDUCE STACKPLOTS

In [ ]:
h_mixed = {}
for region in ["SR", "fakes_CR", "ttbar_CR", "OS_CR_bvetoL"]:
    h_mixed[region] = {}
    for k in aggregated_samples.keys():
        if k == "Fake" or k == "Data":
            h_mixed[region][k] = h_data[region][k]
        else:
            if do_variations == True:
                if k == "WZ":
                    h_mixed[region][k] = h_varied_WZ[region][k]
                else:
                    h_mixed[region][k] = h_varied[region][k]
            else:
                if k == "WZ":
                    h_mixed[region][k] = h_WZ[region][k]
                else:
                    h_mixed[region][k] = h[region][k]
    h_mixed[region]["Fake"] = h_data[region]["Fake"]

In [ ]:
%%capture

regions = ["SR", "fakes_CR", "ttbar_CR", "OS_CR_bvetoL"]

if "stack" not in os.listdir("./" + results_dir):
    os.mkdir("./" + results_dir + "/stack")

if do_variations == True:
    for region in regions:
        for var in variations:
            for var_type in ['up', 'down']:
                for v in variables[region]:
                    if region != "SR":
                        stackplot(region, v, "etau", "./" + results_dir + "/stack", h_mixed, variation = var, var_type = var_type)
                        stackplot(region, v, "mutau", "./" + results_dir +"/stack", h_mixed, variation = var, var_type = var_type) 
                    else:
                        stackplot(region, v, "etau", "./" + results_dir + "/stack", h_mixed, blinded = True, variation = var, var_type = var_type)
                        stackplot(region, v, "mutau", "./" + results_dir + "/stack", h_mixed, blinded = True, variation = var, var_type = var_type)
else:
    for region in regions:
        for v in variables[region]:
            if region != "SR":
                stackplot_no_var(region, v, "etau", "./" + results_dir + "/stack", h_mixed)
                stackplot_no_var(region, v, "mutau", "./" + results_dir +"/stack", h_mixed) 
            else:
                stackplot_no_var(region, v, "etau", "./" + results_dir + "/stack", h_mixed, blinded = True)
                stackplot_no_var(region, v, "mutau", "./" + results_dir + "/stack", h_mixed, blinded = True) 

In [ ]:
import dask

In [ ]:
dask.__version__